<a href="https://colab.research.google.com/github/joimb9064/master_thesis/blob/main/Genetic_Tabu_search_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
from typing import List, Dict, Any, Optional, Tuple  # Added Tuple
import logging
import sys
import time
import random
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.layout import Layout
from rich.live import Live
from rich.text import Text
import logging
import threading
from google.colab import drive
import os
from rich import box  # This is the import we need
from datetime import datetime, timedelta
import csv
import matplotlib.pyplot as plt
import collections
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import functools
import copy
import itertools

# Mount Google Drive
drive.mount('/content/drive')

# Define the log directory in Google Drive
log_dir = "/content/drive/My Drive/EdgeSimPy/logs"

# Create the directory if it does not exist
os.makedirs(log_dir, exist_ok=True)

# Generate a log filename with current date and time
log_filename = datetime.now().strftime("simulation_%Y-%m-%d_%H-%M-%S.log")
full_log_path = os.path.join(log_dir, log_filename)

# List existing log files in the directory (optional)
try:
    print("\n📂 Available log files in EdgeSimPy/logs:")
    for filename in os.listdir(log_dir):
        print(filename)
except Exception as e:
    print(f"Error listing log files: {e}")

# Configure logging to write to both console and file
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG to capture more detailed logs
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),  # Output to console
        logging.FileHandler(full_log_path)  # Save logs to dated file in Google Drive
    ]
)

# Create a logger instance
logger = logging.getLogger(__name__)


class Task:
    def __init__(self,
                 task_id: int,
                 task_type: str,
                 input_size: float,    # Input data size in GB
                 output_size: float,   # Output data size in GB
                 cpu_required: float): # Total MI required
        # Basic identification
        self.id = task_id
        self.type = task_type
        # Resource requirements
        self.input_size = input_size
        self.output_size = output_size
        self.total_cpu_required = cpu_required
        self.remaining_cpu = cpu_required

        # Timing metrics (CloudSim-style)
        self.arrival_time = datetime.now().timestamp()  # Set arrival time immediately
        self.start_time = None    # Will be set when actual processing begins
        self.completion_time = None
        self.exec_start_time = None
        self.actual_exec_time = 0.0     # Initialize as float
        self.turnaround_time = 0.0      # Initialize as float
        self.waiting_time = 0.0         # Initialize as float

        # Status tracking
        self.status = 'CREATED'
        self.completion_percentage = 0.0
        self.assigned_resource = None

        # Progress tracking
        self.transferred_input = 0.0
        self.transferred_output = 0.0
        self.processed_mi = 0.0

        # Tracking flags
        self._time_initialized = False
        self.last_update_time = None
    def calculate_timing_metrics(self):
        """Calculate final timing metrics when task completes"""
        if self.completion_time and self.arrival_time:
            # Total time from arrival to completion
            self.turnaround_time = self.completion_time - self.arrival_time

            # Time spent in actual processing (excluding transfers)
            if self.exec_start_time:
                self.actual_exec_time = self.completion_time - self.exec_start_time
            else:
                self.actual_exec_time = self.completion_time - self.start_time

            # Waiting time is turnaround time minus execution time
            self.waiting_time = self.turnaround_time - self.actual_exec_time

            # Ensure we don't have negative times
            self.turnaround_time = max(0, self.turnaround_time)
            self.actual_exec_time = max(0, self.actual_exec_time)
            self.waiting_time = max(0, self.waiting_time)
    def estimate_execution_time(self, resource) -> float:
        """
        Calculate execution time considering separate input and output transfers
        """
        # Input transfer time (for read tasks)
        input_mb = self.input_size * 1024
        input_transfer_time = input_mb / resource.total_bandwidth if input_mb > 0 else 0

        # Processing time
        processing_time = self.total_cpu_required / resource.total_cpu_rating

        # Output transfer time (for write tasks)
        output_mb = self.output_size * 1024
        output_transfer_time = output_mb / resource.total_bandwidth if output_mb > 0 else 0

        total_time = input_transfer_time + processing_time + output_transfer_time
        return total_time

    def update_progress(self, resource, current_time: float) -> Dict:
        """
        Update task progress with fixed time step calculations and enhanced timing metrics

        This method handles the complete lifecycle of a task, including:
        - Input data transfer
        - Processing
        - Output data transfer
        - Timing metrics calculation
        """
        # Log initial state for debugging
        logger.info(f"Task {self.id} UPDATE: Current state={self.status}, Time={current_time:.2f}")

        # SECTION 1: Initialize timing tracking
        if not self._time_initialized:
            logger.info(f"Task {self.id} INIT: First update at time {current_time:.2f}")
            self._time_initialized = True
            self.last_update_time = current_time

        # SECTION 2: Calculate time steps for this update
        elapsed = current_time - self.last_update_time  # Time since last update
        steps = max(1, int(elapsed / 1.0))  # Divide into 1-second steps
        time_per_step = elapsed / steps     # Actual time per step

        logger.info(f"Task {self.id} STEP: Time since last update={elapsed:.3f}s, Steps={steps}")

        # Calculate bandwidth in GB/s (convert from Mb/s)
        step_bandwidth = (resource.total_bandwidth / 8.0) / 1024.0

        # SECTION 3: State Machine - Handle different task states
        old_status = self.status  # Track state transitions

        if self.status == 'CREATED':
            # Initial state: Set up task for processing
            self.status = 'READY'
            self.transferred_input = 0.0
            self.transferred_output = 0.0
            self.processed_mi = 0.0
            if not self.start_time:  # Only set start_time if not already set
                self.start_time = current_time  # Record when task starts
                logger.info(f"Task {self.id} TRANSITION: CREATED → READY at {current_time:.2f}")

        elif self.status == 'READY':
            # Determine next phase based on input requirements
            if self.input_size > 0:
                self.status = 'TRANSFERRING_INPUT'
                logger.info(f"Task {self.id} TRANSITION: READY → TRANSFERRING_INPUT (input_size={self.input_size}GB)")
            else:
                self.status = 'PROCESSING'
                self.exec_start_time = current_time  # Record processing start
                logger.info(f"Task {self.id} TRANSITION: READY → PROCESSING (no input transfer needed)")
            self.completion_percentage = 0

        elif self.status == 'TRANSFERRING_INPUT':
            # Calculate input data transfer for this time step
            step_transfer = step_bandwidth * time_per_step * steps
            old_transfer = self.transferred_input
            self.transferred_input = min(self.input_size, self.transferred_input + step_transfer)

            logger.info(f"Task {self.id} TRANSFER: Input {old_transfer:.3f}GB → {self.transferred_input:.3f}GB of {self.input_size:.3f}GB")

            if self.transferred_input >= self.input_size:
                # Input transfer complete, start processing
                self.status = 'PROCESSING'
                self.exec_start_time = current_time  # Record actual processing start time
                self.completion_percentage = 0
                logger.info(f"Task {self.id} TRANSITION: TRANSFERRING_INPUT → PROCESSING at {current_time:.2f}")
            else:
                self.completion_percentage = (self.transferred_input / self.input_size) * 100

        elif self.status == 'PROCESSING':
            # Calculate processing progress
            step_processing = resource.total_cpu_rating * time_per_step * steps
            old_processed = self.processed_mi
            self.processed_mi = min(self.total_cpu_required, self.processed_mi + step_processing)
            self.remaining_cpu = self.total_cpu_required - self.processed_mi

            logger.info(f"Task {self.id} PROCESSING: {old_processed:.0f}MI → {self.processed_mi:.0f}MI of {self.total_cpu_required:.0f}MI")

            if self.processed_mi >= self.total_cpu_required:
                # Processing complete, check if output transfer needed
                if self.output_size > 0:
                    self.status = 'TRANSFERRING_OUTPUT'
                    self.completion_percentage = 0
                    logger.info(f"Task {self.id} TRANSITION: PROCESSING → TRANSFERRING_OUTPUT (output_size={self.output_size}GB)")
                else:
                    # No output transfer needed, task is complete
                    self.status = 'COMPLETED'
                    self.completion_time = current_time
                    self.calculate_timing_metrics()
                    self.completion_percentage = 100
                    logger.info(f"Task {self.id} TRANSITION: PROCESSING → COMPLETED at {current_time:.2f}")
            else:
                self.completion_percentage = (self.processed_mi / self.total_cpu_required) * 100

        elif self.status == 'TRANSFERRING_OUTPUT':
            # Calculate output data transfer for this time step
            step_transfer = step_bandwidth * time_per_step * steps
            old_transfer = self.transferred_output
            self.transferred_output = min(self.output_size, self.transferred_output + step_transfer)

            logger.info(f"Task {self.id} TRANSFER: Output {old_transfer:.3f}GB → {self.transferred_output:.3f}GB of {self.output_size:.3f}GB")

            if self.transferred_output >= self.output_size:
                # Output transfer complete, task is finished
                self.status = 'COMPLETED'
                self.completion_time = current_time
                self.calculate_timing_metrics()
                self.completion_percentage = 100
                logger.info(f"Task {self.id} TRANSITION: TRANSFERRING_OUTPUT → COMPLETED at {current_time:.2f}")
            else:
                self.completion_percentage = (self.transferred_output / self.output_size) * 100

        # Log state transition if it occurred
        if old_status != self.status:
            logger.info(f"Task {self.id} STATE CHANGE: {old_status} → {self.status}")

        # SECTION 4: Update timing and return status
        self.last_update_time = current_time

        # Calculate current execution time
        current_exec_time = current_time - (self.start_time or current_time)

        # Final progress report for this update
        logger.info(f"Task {self.id} PROGRESS: Status={self.status}, Completion={self.completion_percentage:.1f}%, ExecTime={current_exec_time:.2f}s")

        return {
            'status': self.status,
            'progress': self.completion_percentage,
            'exec_time': current_exec_time
        }
    def calculate_final_metrics(self):
        """
        Calculate final timing metrics when task completes
        """
        if self.completion_time and self.arrival_time and self.exec_start_time:
            # Turnaround time: time from arrival to completion
            self.turnaround_time = self.completion_time - self.arrival_time

            # Actual execution time: time spent processing (excluding transfers)
            self.actual_exec_time = self.completion_time - self.exec_start_time

            # Waiting time: turnaround time minus actual execution time
            self.waiting_time = self.turnaround_time - self.actual_exec_time
    def update_execution(self, resource, time_step: float) -> bool:
            """
            Process task as a single unit, tracking overall progress.
            Returns True if task is completed.
            """
            if self.status == 'CREATED':
                self.status = 'RUNNING'
                self.start_time = datetime.now().timestamp()

            elif self.status == 'RUNNING':
                # Calculate progress including both transfer and processing
                progress = (time_step * resource.total_cpu_rating)
                self.remaining_cpu -= progress

                if self.remaining_cpu <= 0:
                    self.status = 'COMPLETED'
                    self.completion_time = datetime.now().timestamp()
                    self.execution_time = self.completion_time - self.start_time
                    return True

            return False
    def process(self, available_cpu: float) -> Dict:
        """
        Process the task with available CPU
        Returns processing details
        """
        processed = min(available_cpu, self.remaining_cpu)
        self.remaining_cpu -= processed

        # Log task processing details
        logger.info(f"Processing task {self.id} on resource. Remaining CPU: {self.remaining_cpu}")

        # Calculate completion percentage
        completion_percentage = (self.total_cpu_required - self.remaining_cpu) / self.total_cpu_required * 100

        # Update status
        if self.remaining_cpu <= 0:
            self.status = 'completed'
            self.completion_time = datetime.now().timestamp()

        return {
            'processed': processed,
            'remaining': self.remaining_cpu,
            'status': self.status,
            'completion_percentage': completion_percentage
        }
class Resource:
    def __init__(self,
                 resource_id: int,
                 resource_type: str,
                 cpu_rating: int,    # MIPS
                 memory: int,        # GB
                 bandwidth: int):    # Mb/s
        # Resource identification
        self.id = resource_id
        self.type = resource_type

        # Resource capabilities
        self.total_cpu_rating = cpu_rating
        self.total_memory = memory
        self.total_bandwidth = bandwidth

        # Task tracking
        self.task_queue = []
        self.current_task = None
        self.completed_tasks = []
        self.failed_tasks = []
        self.failed_tasks_count = 0

        # Resource utilization tracking
        self.used_cpu = 0
        self.used_memory = 0
        self.current_load = 0.0

    def calculate_resource_utilization(self):
        """
        Debug resource utilization
        """
        # First, log the current state
        task_info = "No task"
        if self.current_task:
            task_info = f"Task {self.current_task.id} ({self.current_task.type}) in state {self.current_task.status}"
            # Add extensive task state logging
            logger.info(f"""
            CURRENT TASK STATE:
            Task ID: {self.current_task.id}
            Type: {self.current_task.type}
            Status: {self.current_task.status}
            Progress: {self.current_task.completion_percentage}%
            Input Size: {self.current_task.input_size} GB
            Output Size: {self.current_task.output_size} GB
            Processed MI: {self.current_task.processed_mi} / {self.current_task.total_cpu_required}
            Input Transfer: {self.current_task.transferred_input} / {self.current_task.input_size} GB
            Output Transfer: {self.current_task.transferred_output} / {self.current_task.output_size} GB
            """)

        logger.info(f"Resource {self.id} ({self.type}) - {task_info}")

        # Default values
        cpu_util = 0
        mem_util = 0
        bw_util = 0

        # Super simple utilization based purely on state
        if self.current_task:
            status = self.current_task.status

            # Debug status transitions
            logger.info(f"Current status: {status}")

            if status == 'PROCESSING':
                cpu_util = 100  # Full CPU during processing
                logger.info(f"Setting CPU utilization to 100% for PROCESSING state")

            if status in ['TRANSFERRING_INPUT', 'TRANSFERRING_OUTPUT']:
                bw_util = 100  # Full bandwidth during transfers
                logger.info(f"Setting bandwidth utilization to 100% for {status}")

            # Set memory utilization based on data presence
            if self.current_task.input_size > 0 or self.current_task.output_size > 0:
                mem_util = 50  # Using 50% as a simple indicator
                logger.info(f"Setting memory utilization to 50% due to data presence")

        # Log final values
       # logger.info(f"""
       # FINAL UTILIZATION VALUES:
       #CPU: {cpu_util}%
       # Memory: {mem_util}%
       # Bandwidth: {bw_util}%
       # """)

        return {
            'cpu_utilization': cpu_util,
            'memory_utilization': mem_util,
            'bandwidth_utilization': bw_util,
            'overall_utilization': (cpu_util + mem_util + bw_util) / 3,
            'raw_cpu_usage': (cpu_util / 100.0) * self.total_cpu_rating,
            'raw_memory_usage': (mem_util / 100.0) * self.total_memory * 1024,
            'raw_bandwidth_usage': (bw_util / 100.0) * self.total_bandwidth,
            'active_tasks': 1 if self.current_task else 0,
            'transfer_phase': self.current_task.status if self.current_task else 'NONE'
        }
    def _get_zero_utilization(self):
        """Helper method to return zero utilization state"""
        return {
            'cpu_utilization': 0,
            'memory_utilization': 0,
            'bandwidth_utilization': 0,
            'overall_utilization': 0,
            'raw_cpu_usage': 0,
            'raw_memory_usage': 0,
            'raw_bandwidth_usage': 0,
            'active_tasks': 0,
            'transfer_phase': 'NONE'
        }
    def can_process_task(self, task: Task) -> Tuple[bool, str]:
        """
        Check if resource can process a given task based on task type and resource type.
        Only RT1 and RT3 tasks are restricted from Smartphone and Raspberry Pi resources.
        """
        # Cloud resources can process all tasks
        if self.type.startswith("Cloud_"):
            return True, ""

        # For Smartphone and Raspberry Pi resources
        if self.type.startswith(("Smartphone_", "Raspberry_")):
            return True, ""

        # All other combinations are valid
        return True, ""

    def process_task(self, resource, current_time: float) -> Dict:
        """
        Process task as a single unit, ensuring proper completion.
        """
        status = {
            'processed': False,
            'completed': False,
            'task_id': self.id,
            'progress': self.completion_percentage
        }

        if self.status == 'CREATED':
            self.status = 'RUNNING'
            self.start_time = current_time
            status['processed'] = True

        elif self.status == 'RUNNING':
            # Calculate progress including both transfer and processing
            time_step = current_time - self.start_time
            progress = (time_step * resource.total_cpu_rating)
            self.remaining_cpu = max(0, self.remaining_cpu - progress)

            # Update completion percentage
            self.completion_percentage = min(100, ((self.total_cpu_required - self.remaining_cpu) /
                                                self.total_cpu_required * 100))

            if self.remaining_cpu <= 0:
                self.status = 'COMPLETED'
                self.completion_time = current_time
                self.actual_exec_time = self.completion_time - self.start_time
                status['completed'] = True

            status['processed'] = True
            status['progress'] = self.completion_percentage

        return status
    def enqueue_task(self, task: Task):
        """Add task to queue and update resource utilization"""
        can_process, failure_reason = self.can_process_task(task)

        if can_process:
            task.status = 'READY'
            self.task_queue.append(task)
            logger.info(f"Task {task.id} queued on {self.type}. Queue length: {len(self.task_queue)}")
        else:
            task.status = 'FAILED'
            task.failure_reason = failure_reason
            self.failed_tasks.append(task)
            self.failed_tasks_count += 1
            logger.warning(f"Task {task.id} ({task.type}) failed on {self.type}: {failure_reason}")
    def process_queue(self, current_time: float) -> Dict:
        """
        Process tasks in queue with fixed progress tracking
        """
        status = {
            'completed_tasks': len(self.completed_tasks),
            'current_task': None,
            'queue_length': len(self.task_queue),
            'resource_utilization': self.calculate_resource_utilization()
        }

        if self.current_task:
            # Update task progress
            progress = self.current_task.update_progress(self, current_time)

            # Update status dictionary
            status['current_task'] = {
                'id': self.current_task.id,
                'type': self.current_task.type,
                'phase': self.current_task.status,
                'progress': progress['progress'],
                'input_size': self.current_task.input_size,
                'output_size': self.current_task.output_size
            }

            # Handle task completion
            if self.current_task.status == 'COMPLETED':
                self.completed_tasks.append(self.current_task)
                logger.info(f"Task {self.current_task.id} completed on {self.type}")
                self.current_task = None

        # Start new task if available
        if not self.current_task and self.task_queue:
            self.current_task = self.task_queue.pop(0)
            # Make sure tasks are in CREATED state before processing
            logger.info(f"Starting Task {self.current_task.id} (status: {self.current_task.status}) - forcing to CREATED")
            self.current_task.status = 'CREATED'

            self.current_task.exec_start_time = current_time
            self.current_task._time_initialized = False  # Reset this flag to trigger initialization

        return status
class ResourceFocusedScheduler:
    """
    Scheduler with resource-focused real-time visualization
    """
    def __init__(self, resources: List[Resource]):
        self.resources = resources
        self.current_time = 0
        #self.arrival_rate = 1.0  # Default task arrival rate
        self.console = Console()
        self.metrics = {
            'total_tasks': 0,
            'completed_tasks': 0,
            'failed_tasks': 0,
            'globally_failed_tasks': [],  # Store globally unassigned failed tasks
            'makespan': 0,
            'throughput': 0
        }
    def calculate_turnaround_time(self, task, simulation_start_time):
        """
        Calculate turnaround time relative to simulation start time
        """
        if (task.arrival_time is not None and
            task.completion_time is not None):
            # Adjust times relative to simulation start
            arrival_relative = task.arrival_time - simulation_start_time
            completion_relative = task.completion_time - simulation_start_time

            turnaround_time = max(completion_relative - arrival_relative, 0)
            return turnaround_time
        return 0.0

    def calculate_waiting_time(self, task):
        """
        Calculate waiting time
        """
        if (task.turnaround_time is not None and
            task.actual_exec_time is not None):
            waiting_time = max(task.turnaround_time - task.actual_exec_time, 0)
            return waiting_time
        return 0.0

    def calculate_timing_metrics(self, completed_tasks: List[Task], simulation_start_time: float) -> Dict:
        """
        Calculate timing metrics with guaranteed dictionary return
        """
        # Initialize metrics with default values
        metrics = {
            'average_turnaround_time': 0.0,
            'average_waiting_time': 0.0,
            'average_execution_time': 0.0,
            'total_tasks_processed': 0,
            'total_turnaround_time': 0.0,
            'total_waiting_time': 0.0,
            'total_execution_time': 0.0
        }

        if not completed_tasks:
            logger.warning("No completed tasks to process")
            return metrics

        total_turnaround = 0.0
        total_waiting = 0.0
        total_execution = 0.0
        valid_tasks = 0

        logger.info(f"\nProcessing timing metrics for {len(completed_tasks)} completed tasks")

        for task in completed_tasks:
            if task.status == 'COMPLETED':
                # Log timing values for debugging
                logger.debug(f"""
                Task {task.id} timing values:
                - Arrival: {task.arrival_time}
                - Start: {task.start_time}
                - Exec Start: {task.exec_start_time}
                - Completion: {task.completion_time}
                """)

                # Ensure timing values are valid
                if all(x is not None for x in [task.arrival_time, task.start_time,
                                            task.completion_time, task.exec_start_time]):
                    # Recalculate metrics to ensure consistency
                    task.turnaround_time = max(0, task.completion_time - task.arrival_time)
                    task.actual_exec_time = max(0, task.completion_time - task.exec_start_time)
                    task.waiting_time = max(0, task.turnaround_time - task.actual_exec_time)

                    if task.turnaround_time > 0 and task.actual_exec_time > 0:
                        total_turnaround += task.turnaround_time
                        total_waiting += task.waiting_time
                        total_execution += task.actual_exec_time
                        valid_tasks += 1
                        logger.info(f"Task {task.id} metrics valid - turnaround: {task.turnaround_time:.2f}s")
                    else:
                        logger.warning(f"Task {task.id} has zero or negative timing values")
                else:
                    logger.warning(f"Task {task.id} has missing timing values")

        # Update metrics if we have valid tasks
        if valid_tasks > 0:
            metrics.update({
                'average_turnaround_time': total_turnaround / valid_tasks,
                'average_waiting_time': total_waiting / valid_tasks,
                'average_execution_time': total_execution / valid_tasks,
                'total_tasks_processed': valid_tasks,
                'total_turnaround_time': total_turnaround,
                'total_waiting_time': total_waiting,
                'total_execution_time': total_execution
            })

        # Log final summary
        logger.info(f"""
        Timing Metrics Summary:
        - Total Tasks: {len(completed_tasks)}
        - Valid Tasks: {valid_tasks}
        - Average Turnaround: {metrics['average_turnaround_time']:.2f}s
        - Average Waiting: {metrics['average_waiting_time']:.2f}s
        - Average Execution: {metrics['average_execution_time']:.2f}s
        """)

        return metrics
    def calculate_datacenter_utilization(self, start_time: float, end_time: float) -> Dict:
        """
        Calculate utilization for different datacenter types following the paper's integral approach
        """
        # Separate resources by datacenter type
        smartphone_resources = [r for r in self.resources if r.type.startswith('Smartphone_')]
        raspberry_pi_resources = [r for r in self.resources if r.type.startswith('Raspberry_')]
        cloud_resources = [r for r in self.resources if r.type.startswith('Cloud_')]

        def calculate_datacenter_ru(resources: List[Resource]) -> Dict:
            """
            Calculate resource utilization for a specific datacenter type
            """
            # Collect individual resource utilizations
            resource_utilizations = [r.calculate_resource_utilization() for r in resources]

            # Calculate datacenter-wide metrics
            return {
                'total_resources': len(resources),
                'avg_cpu_utilization': np.mean([ru['cpu_utilization'] for ru in resource_utilizations]),
                'avg_memory_utilization': np.mean([ru['memory_utilization'] for ru in resource_utilizations]),
                'avg_bandwidth_utilization': np.mean([ru['bandwidth_utilization'] for ru in resource_utilizations]),
                'overall_utilization': np.mean([ru['overall_utilization'] for ru in resource_utilizations]),
                'active_tasks': sum(ru['active_tasks'] for ru in resource_utilizations),
                'raw_metrics': resource_utilizations
            }

        # Calculate utilization for each datacenter type
        datacenter_utilization = {
            'smartphone_edge': calculate_datacenter_ru(smartphone_resources),
            'raspberry_pi_edge': calculate_datacenter_ru(raspberry_pi_resources),
            'cloud': calculate_datacenter_ru(cloud_resources)
        }

        # Calculate overall datacenter utilization
        datacenter_utilization['overall'] = {
            'total_resources': len(self.resources),
            'total_simulation_time': end_time - start_time,
            'avg_cpu_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_cpu_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_cpu_utilization'],
                datacenter_utilization['cloud']['avg_cpu_utilization']
            ]),
            'avg_memory_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_memory_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_memory_utilization'],
                datacenter_utilization['cloud']['avg_memory_utilization']
            ]),
            'avg_bandwidth_utilization': np.mean([
                datacenter_utilization['smartphone_edge']['avg_bandwidth_utilization'],
                datacenter_utilization['raspberry_pi_edge']['avg_bandwidth_utilization'],
                datacenter_utilization['cloud']['avg_bandwidth_utilization']
            ]),
            'total_active_tasks': sum(
                datacenter_utilization[dc_type]['active_tasks']
                for dc_type in ['smartphone_edge', 'raspberry_pi_edge', 'cloud']
            )
        }

        return datacenter_utilization

    def generate_tasks(self, total_tasks: int) -> Tuple[List[Task], List[float], List[float]]:
            """
            Generate tasks with Poisson-distributed arrival times and return raw inter-arrival times.

            Args:
                total_tasks (int): The total number of tasks to generate.

            Returns:
                Tuple[List[Task], List[float], List[float]]: A tuple containing:
                    - List of generated Task objects
                    - List of cumulative arrival times
                    - List of raw inter-arrival times
            """


            simulation_duration = 100
            lambda_rate = total_tasks / simulation_duration
            inter_arrival_times = []
            cumulative_times = [0]  # Start with 0 as the first arrival time

            while len(inter_arrival_times) < total_tasks - 1:  # We need one less inter-arrival time than total tasks
                interval = np.random.exponential(1.0 / lambda_rate)
                inter_arrival_times.append(interval)
                cumulative_times.append(cumulative_times[-1] + interval)
            # Use a base time to ensure consistent and positive arrival times
            # In the generate_tasks method
            base_time = datetime.now().timestamp()

            tasks = []
            for i in range(total_tasks):
                task = Task(
                    task_id=i + 1,
                    task_type="placeholder",
                    input_size=0.0,    # Changed from data_size to input_size
                    output_size=0.0,   # Added output_size
                    cpu_required=0.0
                )
                task.arrival_time = base_time + cumulative_times[i]
                tasks.append(task)

            # Write inter-arrival times to a CSV file
            with open('inter_arrival_times.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Inter-arrival Time'])
                for time in inter_arrival_times:
                    writer.writerow([time])

            return tasks, cumulative_times, inter_arrival_times

    def _create_resource_panel(self, resource: Resource, status: Dict) -> Panel:
        """
        Create a detailed panel for a specific resource with comprehensive task information.
        """
        table = Table(show_header=False, show_lines=True)

        # Resource basic information
        table.add_row("[bold]Resource Details[/bold]")
        table.add_row(f"[cyan]Type:[/cyan] {resource.type}")
        table.add_row(f"[green]CPU Rating:[/green] {resource.total_cpu_rating} MI/s")
        table.add_row(f"[blue]Memory:[/blue] {resource.total_memory} GB")
        table.add_row(f"[yellow]Bandwidth:[/yellow] {resource.total_bandwidth} MB/s")

        # Utilization information
        table.add_row("\n[bold]Utilization Metrics[/bold]")
        table.add_row(
            f"[green]CPU Usage:[/green] {status['resource_utilization']['cpu_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_cpu_usage']:.2f}/{resource.total_cpu_rating} MI/s)"
        )
        table.add_row(
            f"[blue]Memory Usage:[/blue] {status['resource_utilization']['memory_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_memory_usage']:.2f}/{resource.total_memory} GB)"
        )
        table.add_row(
            f"[yellow]Bandwidth Usage:[/yellow] {status['resource_utilization']['bandwidth_utilization']:.2f}% "
            f"({status['resource_utilization']['raw_bandwidth_usage']:.2f}/{resource.total_bandwidth} MB/s)"
        )

        # Task Queue Information
        table.add_row("\n[bold]Task Queue[/bold]")
        table.add_row(f"[yellow]Queued Tasks:[/yellow] {status['queue_length']}")

        # Current Tasks
        table.add_row("\n[bold]Current Tasks[/bold]")
        if status['current_task']:
            current_task = status['current_task']
            table.add_row(
                f"[blue]Task {current_task['id']} ({current_task['type']}):[/blue] "
                f"Phase: {current_task['phase']} Progress: {current_task['progress']}"
            )
        else:
            table.add_row("[dim]No tasks currently processing[/dim]")

        # Failed Tasks
        table.add_row("\n[bold]Failed Tasks[/bold]")
        if resource.failed_tasks:
            for task in resource.failed_tasks:
                table.add_row(
                    f"[red]Task {task.id} ({task.type}):[/red] "
                    f"Reason: {task.failure_reason}"
                )
        else:
            table.add_row("[dim]No failed tasks[/dim]")

        return Panel(
            table,
            title=f"Resource {resource.id}: {resource.type}",
            border_style="green"
        )
    def verify_and_fix_task_distribution(self, total_tasks: int) -> List[Task]:
        """
        Emergency fix to directly ensure tasks are properly queued to resources
        regardless of distribution algorithm issues.
        """
        logger.info("🚨 EMERGENCY FIX: Verifying task distribution...")

        # Check current queue status
        total_queued = sum(len(resource.task_queue) for resource in self.resources)
        logger.info(f"Current queued tasks: {total_queued} of {total_tasks}")

        # If we have tasks queued, no action needed
        if total_queued >= total_tasks:
            logger.info("✅ Task distribution verified: All tasks are properly queued")

            # Collect all tasks to return
            all_tasks = []
            for resource in self.resources:
                all_tasks.extend(resource.task_queue)
                all_tasks.extend(resource.failed_tasks)

            return all_tasks

        # If we have zero or insufficient tasks queued, perform emergency distribution
        logger.warning(f"⚠️ Task distribution problem detected: Only {total_queued} of {total_tasks} tasks queued")

        # Clear all existing resource queues
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Generate fresh tasks
        logger.info("Generating emergency replacement tasks...")
        tasks, _, _ = self.generate_tasks(total_tasks)

        # Define task types with fixed requirements
        task_types = [
            # Read Tasks
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Distribute tasks using simple round-robin
        distributed_tasks = []
        resource_index = 0

        for i, task_record in enumerate(tasks):
            # Select task type based on index
            task_type = task_types[i % len(task_types)]

            # Create task with proper properties
            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time
            task.status = 'CREATED'  # Ensure correct initial status

            # Assign to resource using round-robin
            resource = self.resources[resource_index]
            resource.task_queue.append(task)

            # Log the assignment
            logger.info(f"Emergency assigned Task {task.id} ({task.type}) to {resource.type}")

            # Advance resource index for round-robin
            resource_index = (resource_index + 1) % len(self.resources)

            # Add to distributed tasks list
            distributed_tasks.append(task)

        # Verify fix worked
        total_queued = sum(len(resource.task_queue) for resource in self.resources)
        logger.info(f"Emergency fix completed: {total_queued} of {total_tasks} tasks now queued")

        # Double check each resource has tasks
        for resource in self.resources:
            logger.info(f"Resource {resource.type} now has {len(resource.task_queue)} tasks queued")

        return distributed_tasks

    def calculate_load_balancing_metrics(self) -> Dict:
        """
        Calculate load balancing metrics based on resource utilization formula:
        1. U(Ri) = L(Ri) / Makespan - Utilization of each resource
        2. Uav = (Σ U(Ri)) / n - Average utilization across all resources

        Returns:
            Dict: Load balancing metrics
        """
        # Start debug logging
        logger.info("===== LOAD BALANCING METRICS CALCULATION STARTED =====")

        # Get makespan from metrics (correct order - define before using)
        # Calculate makespan dynamically
        makespan = max(
            max(
                (task.completion_time or 0)
                for resource in self.resources
                for task in resource.completed_tasks
            ),
            0.001  # Ensure a minimum makespan to avoid division by zero
        )

        # Debug output
        logger.info(f"DEBUG - Makespan value: {makespan}")
        logger.info(f"DEBUG - self.metrics contents: {self.metrics}")

        # Check for valid makespan
        if makespan <= 0:
            logger.warning("Makespan is zero or negative, returning empty metrics")
            return {
                'resource_utilizations': {},
                'average_utilization': 0,
                'load_balance_score': 0,
                'utilization_std_dev': 0,
                'resource_type_metrics': {}
            }

        # Calculate load on each resource
        # L(Ri) = total processing time of all tasks assigned to Ri
        resource_loads = {}
        resource_utilizations = {}

        # Group resources by type
        smartphone_resources = []
        raspberry_pi_resources = []
        cloud_resources = []

        # Check and fix task timing issues
        task_timing_issues = 0
        fixed_task_count = 0

        # Process each resource
        logger.info("Processing individual resource metrics:")

        for resource in self.resources:
            # Group resource by type
            if resource.type.startswith('Smartphone_'):
                smartphone_resources.append(resource)
            elif resource.type.startswith('Raspberry_'):
                raspberry_pi_resources.append(resource)
            elif resource.type.startswith('Cloud_'):
                cloud_resources.append(resource)

            # Fix any task timing issues
            for task in resource.completed_tasks:
                if not hasattr(task, 'actual_exec_time') or task.actual_exec_time is None:
                    task_timing_issues += 1
                    # Try to calculate from raw timestamps
                    if hasattr(task, 'completion_time') and hasattr(task, 'exec_start_time'):
                        if task.completion_time and task.exec_start_time:
                            task.actual_exec_time = max(0.001, task.completion_time - task.exec_start_time)
                            fixed_task_count += 1
                            logger.info(f"Fixed task {task.id} exec_time to {task.actual_exec_time:.4f}s")
                elif task.actual_exec_time <= 0:
                    task_timing_issues += 1
                    # Force a minimum value
                    task.actual_exec_time = 0.001  # 1 millisecond minimum
                    fixed_task_count += 1
                    logger.info(f"Corrected zero/negative exec_time for task {task.id}")

            # Debug task information
            completed_task_count = len(resource.completed_tasks)
            valid_task_count = sum(
                1 for task in resource.completed_tasks
                if hasattr(task, 'actual_exec_time') and task.actual_exec_time is not None and task.actual_exec_time > 0
            )

            logger.info(f"Resource {resource.id} ({resource.type}):")
            logger.info(f"  - Completed tasks: {completed_task_count}")
            logger.info(f"  - Tasks with valid execution time: {valid_task_count}")

            # Log individual task execution times
            task_times_summary = []
            for i, task in enumerate(resource.completed_tasks[:5]):  # Log first 5 tasks for brevity
                task_exec_time = getattr(task, 'actual_exec_time', None)
                task_times_summary.append(f"Task {task.id}: {task_exec_time:.4f}s" if task_exec_time else f"Task {task.id}: None")

            logger.info(f"  - Task times: {', '.join(task_times_summary)}")

            if completed_task_count > 5:
                logger.info(f"  - ... and {completed_task_count - 5} more tasks")

            # Calculate total processing time for all completed tasks
            total_processing_time = sum(
                task.actual_exec_time
                for task in resource.completed_tasks
                if hasattr(task, 'actual_exec_time') and task.actual_exec_time is not None and task.actual_exec_time > 0
            )

            logger.info(f"  - Total processing time: {total_processing_time:.4f} seconds")

            # Calculate utilization: U(Ri) = L(Ri) / Makespan
            utilization = total_processing_time / makespan if makespan > 0 else 0
            logger.info(f"  - Utilization (U(Ri) = L(Ri) / Makespan): {utilization:.4f}")

            resource_loads[resource.id] = total_processing_time
            resource_utilizations[resource.id] = utilization

        # Log task timing fixes
        logger.info(f"Found {task_timing_issues} tasks with timing issues, fixed {fixed_task_count}")

        # Check if we have any valid utilization data
        if not resource_utilizations or all(util == 0 for util in resource_utilizations.values()):
            logger.warning("All resources have zero utilization, metrics will be zero")

        # Calculate average utilization overall: Uav = (Σ U(Ri)) / n
        average_utilization = sum(resource_utilizations.values()) / len(self.resources) if self.resources else 0
        logger.info(f"Average utilization across all resources (Uav): {average_utilization:.4f}")

        # Calculate standard deviation of utilizations (measure of balance)
        utilization_values = list(resource_utilizations.values())
        logger.info(f"All utilization values: {utilization_values}")

        utilization_std_dev = np.std(utilization_values) if utilization_values else 0
        logger.info(f"Standard deviation of utilizations: {utilization_std_dev:.4f}")

        # Load balance score (lower is better - 0 is perfect balance)
        # Using coefficient of variation to normalize
        if average_utilization <= 0.0001:  # Very small threshold
            logger.warning("Average utilization near zero, setting load balance score to 0")
            load_balance_score = 0
        else:
            load_balance_score = utilization_std_dev / average_utilization

        logger.info(f"Load balance score (CV = std_dev / avg): {load_balance_score:.4f}")

        # Calculate metrics per resource type
        def calculate_resource_type_metrics(resources, type_name):
            logger.info(f"\nCalculating metrics for {type_name} resources:")

            if not resources:
                logger.info(f"  No {type_name} resources found")
                return {
                    'average_utilization': 0,
                    'utilization_std_dev': 0,
                    'load_balance_score': 0,
                    'resource_count': 0
                }

            logger.info(f"  Found {len(resources)} {type_name} resources")
            type_utilizations = [resource_utilizations[r.id] for r in resources]
            logger.info(f"  Utilization values: {type_utilizations}")

            type_avg_util = sum(type_utilizations) / len(resources)
            logger.info(f"  Average utilization: {type_avg_util:.4f}")

            type_std_dev = np.std(type_utilizations) if len(type_utilizations) > 1 else 0
            logger.info(f"  Standard deviation: {type_std_dev:.4f}")

            type_balance_score = type_std_dev / type_avg_util if type_avg_util > 0 else 0
            logger.info(f"  Balance score: {type_balance_score:.4f}")

            return {
                'average_utilization': type_avg_util,
                'utilization_std_dev': type_std_dev,
                'load_balance_score': type_balance_score,
                'resource_count': len(resources)
            }

        # Get metrics for each resource type
        logger.info("\nCalculating metrics by resource type:")
        resource_type_metrics = {
            'smartphone': calculate_resource_type_metrics(smartphone_resources, "smartphone"),
            'raspberry_pi': calculate_resource_type_metrics(raspberry_pi_resources, "raspberry_pi"),
            'cloud': calculate_resource_type_metrics(cloud_resources, "cloud")
        }

        # Calculate inter-type balance
        logger.info("\nCalculating inter-type balance metrics:")
        type_avg_utils = [
            resource_type_metrics['smartphone']['average_utilization'],
            resource_type_metrics['raspberry_pi']['average_utilization'],
            resource_type_metrics['cloud']['average_utilization']
        ]
        logger.info(f"  Resource type average utilizations: {type_avg_utils}")

        inter_type_std_dev = np.std(type_avg_utils)
        logger.info(f"  Inter-type standard deviation: {inter_type_std_dev:.4f}")

        inter_type_avg = sum(type_avg_utils) / 3 if sum(type_avg_utils) > 0 else 0
        logger.info(f"  Inter-type average: {inter_type_avg:.4f}")

        inter_type_balance = inter_type_std_dev / inter_type_avg if inter_type_avg > 0 else 0
        logger.info(f"  Inter-type balance score: {inter_type_balance:.4f}")

        resource_type_metrics['inter_type_balance'] = {
            'standard_deviation': inter_type_std_dev,
            'balance_score': inter_type_balance
        }

        logger.info("===== LOAD BALANCING METRICS CALCULATION COMPLETED =====")

        # Force log flush
        for handler in logger.handlers:
            if isinstance(handler, logging.FileHandler):
                handler.flush()

        return {
            'resource_utilizations': resource_utilizations,
            'average_utilization': average_utilization,
            'load_balance_score': load_balance_score,
            'utilization_std_dev': utilization_std_dev,
            'resource_type_metrics': resource_type_metrics
        }

    def _get_edge_cloud_distribution(self, solution):
        """Get distribution of tasks between edge and cloud resources"""
        cloud_tasks = 0
        edge_tasks = 0

        for task, resource in solution.items():
            if resource:  # Skip unassigned tasks
                if resource.type.startswith("Cloud_"):
                    cloud_tasks += 1
                else:  # Edge resources (Raspberry Pi, Smartphone, etc.)
                    edge_tasks += 1

        total_assigned = cloud_tasks + edge_tasks

        if total_assigned > 0:
            cloud_percentage = (cloud_tasks / total_assigned) * 100
            edge_percentage = (edge_tasks / total_assigned) * 100
        else:
            cloud_percentage = 0
            edge_percentage = 0

        return {
            "edge_tasks": edge_tasks,
            "cloud_tasks": cloud_tasks,
            "edge_percentage": edge_percentage,
            "cloud_percentage": cloud_percentage
        }

    def genetic_modified(self, total_tasks: int) -> List[Task]:
        """
        Pure Genetic Algorithm for task scheduling (extracted from the hybrid approach).
        This version focuses solely on genetic algorithm optimization without Tabu Search.

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        # Log start time and show progress indicator
        start_time = time.time()
        logger.info(f"\n⏳ Starting Genetic Algorithm for {total_tasks} tasks...")
        logger.info("Phase 1/3: Task generation and initialization...")

        # Initialize CSV tracking for algorithm progress
        csv_folder = "/content/drive/My Drive/EdgeSimPy/algorithm_tracking"
        os.makedirs(csv_folder, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        ga_csv_path = os.path.join(csv_folder, f'ga_evolution_{timestamp}.csv')
        solution_csv_path = os.path.join(csv_folder, f'solution_comparison_{timestamp}.csv')

        # Initialize CSV files with headers
        with open(ga_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Generation', 'Best_Fitness', 'Average_Fitness', 'Population_Diversity',
                            'Mutation_Rate', 'Crossover_Rate', 'Tasks_Per_Resource',
                            'Load_Balance_Score', 'Solution_Hash'])

        with open(solution_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Algorithm_Phase', 'Timestamp', 'Solution_Cost', 'Load_Balance_Score',
                            'Resource_Distribution', 'Task_Type_Distribution', 'Average_Execution_Time',
                            'Worst_Task_Time', 'Best_Task_Time'])

        try:
            # === STEP 1: Generate tasks with batch processing for efficiency ===
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                } for resource in self.resources
            }

            # Helper methods for tracking metrics
            def _calculate_population_diversity(population):
                """Calculate population diversity as percentage of different assignments"""
                if len(population) <= 1:
                    return 0

                # Take the first solution as reference
                reference = population[0]
                total_assignments = len(reference) * (len(population) - 1)
                different_assignments = 0

                # Compare each solution with the reference
                for solution in population[1:]:
                    for task in reference:
                        if task in solution and reference[task] != solution[task]:
                            different_assignments += 1

                return (different_assignments / total_assignments) * 100 if total_assignments > 0 else 0

            def _get_resource_distribution(solution):
                """Get distribution of tasks per resource"""
                distribution = {resource.type: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if resource:
                        distribution[resource.type] += 1

                return distribution

            def _get_task_type_distribution(solution):
                """Get distribution of task types per resource type"""
                distribution = {}

                for task, resource in solution.items():
                    if resource:
                        resource_type = resource.type.split('_')[0]  # Cloud, Smartphone, or Raspberry
                        task_type = task.type

                        if resource_type not in distribution:
                            distribution[resource_type] = {}

                        if task_type not in distribution[resource_type]:
                            distribution[resource_type][task_type] = 0

                        distribution[resource_type][task_type] += 1

                return distribution

            def fast_exec_time_estimate(task, resource_id):
                """Simplified execution time estimation for speed"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Basic estimate combining transfer and processing time
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']
                process_time = task.total_cpu_required / resource['cpu_rating']


                # Simplified total time calculation
                return transfer_time + process_time

            def simplified_fitness(solution):
                """
                Fitness function based solely on equation (1) from page 4716 of the paper:
                minimize (1/m) * Σ(j=1 to m) Cmax(j)

                Where:
                - m: Number of resources/virtual machines
                - Cmax(j): Maximum completion time (makespan) for resource j

                This minimizes the mean makespan across all resources to achieve
                load balancing and overall system performance.

                Args:
                    solution: Dict mapping tasks to resources

                Returns:
                    float: Fitness value (lower is better)
                """
                # Return high penalty for empty solutions
                if not solution:
                    return float('inf')

                # Get total number of resources/VMs (m)
                m = len(self.resources)

                # Track completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                # Assign tasks and calculate completion times for each resource
                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:  # Skip unassigned tasks
                        continue

                    # Calculate task execution time
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Task starts at either its arrival time or when resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + total_task_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Calculate individual resource makespans (Cmax(j) for each j)
                resource_makespans = list(resource_completion_times.values())

                # Calculate mean makespan as per equation (1): (1/m) * Σ(j=1 to m) Cmax(j)
                mean_makespan = sum(resource_makespans) / m if m > 0 else float('inf')

                # Add penalty only for unassigned tasks
                unassigned_count = len([1 for _, resource in solution.items() if not resource])
                if unassigned_count > 0:
                    mean_makespan += unassigned_count * 10000

                # The fitness is purely the mean makespan without any additional penalties
                return mean_makespan

            # Faster task creation with larger batch size
            batch_size = 500
            initial_tasks = []
            resource_loads = {r.id: 0 for r in self.resources}

            # Show progress for task creation
            logger.info(f"Creating {total_tasks} tasks in batches of {batch_size}...")

            # Define the TASK_TYPES dictionary
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Use random selection for task type
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # Special handling for very small task counts
            if total_tasks <= 10:
                logger.info(f"Small task count ({total_tasks}) detected - using specialized optimization parameters")
                # Smaller population, fewer generations for tiny problems
                population_size = max(5, min(10, total_tasks))
                generations = 3
                max_stagnation = 2
            else:
                # === STEP 2: Adjusted optimization parameters for larger task counts ===
                population_size = min(50, int(total_tasks * 0.1))  # Proportional to total tasks
                generations = min(20, int(total_tasks * 0.05))     # More generations for larger problems
                max_stagnation = 10    # Increased early stopping patience

            # Initialize GA parameters
            # === Initialize epsilon decay parameters ===
            epsilon_start = 1.0  # Start with full exploration
            epsilon_end = 0.1    # End with minimal exploration
            ga_epsilon_decay_rate = (epsilon_start - epsilon_end) / generations
            current_ga_epsilon = epsilon_start

            # With these adaptive mutation parameters
            base_mutation_rate = 0.15
            min_mutation_rate = 0.1
            max_mutation_rate = 0.4
            current_mutation_rate = base_mutation_rate
            diversity_threshold_low = 15  # Percentage below which to increase mutation
            diversity_threshold_high = 40  # Percentage above which to decrease mutation
            crossover_rate = 0.8
            elite_count = 2

            # === STEP 3: Create random initial solution ===
            logger.info("Phase 2/3: Creating initial population...")

            # Create initial population
            population = []
            for i in range(population_size):
                # Create completely random solution
                solution = {}
                for task in initial_tasks:
                    solution[task] = random.choice(self.resources)

                population.append(solution)

            # === STEP 4: Run Optimized Genetic Algorithm ===
            logger.info("Phase 3/3: Running genetic algorithm optimization...")

            # Evaluate initial population
            fitnesses = [simplified_fitness(sol) for sol in population]

            # Track best solution
            best_idx = fitnesses.index(min(fitnesses))
            best_solution = copy.deepcopy(population[best_idx])
            best_fitness = fitnesses[best_idx]
            best_cost = simplified_fitness(best_solution)

            # Record initial state for the solution comparison CSV
            initial_resource_distribution = json.dumps(_get_resource_distribution(best_solution))
            initial_task_type_distribution = json.dumps(_get_task_type_distribution(best_solution))
            initial_task_times = [fast_exec_time_estimate(task, resource.id) for task, resource in best_solution.items()]
            avg_time = sum(initial_task_times) / len(initial_task_times) if initial_task_times else 0
            worst_time = max(initial_task_times) if initial_task_times else 0
            best_time = min(initial_task_times) if initial_task_times else 0

            # Calculate initial load balance
            load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id]) for res in self.resources])

            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Initial_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    initial_resource_distribution,  # Resource distribution
                    initial_task_type_distribution,  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])
            # Add edge-cloud distribution logging here
            ga_edge_cloud_dist = self._get_edge_cloud_distribution(best_solution)
            logger.info(f"GA Solution - Edge: {ga_edge_cloud_dist['edge_percentage']:.2f}%, Cloud: {ga_edge_cloud_dist['cloud_percentage']:.2f}%")

            # Run genetic algorithm with progress updates
            stagnation_counter = 0

            for gen in range(generations):
                # Progress indicator
                current_cost = simplified_fitness(best_solution)
                logger.info(f"Generation {gen+1}/{generations} - Current best cost: {current_cost}")

                # Update epsilon for this generation
                current_ga_epsilon = max(epsilon_end, epsilon_start - gen * ga_epsilon_decay_rate)
                logger.info(f"Current GA epsilon: {current_ga_epsilon:.3f}")

                # Sort by fitness and retain elites
                sorted_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i])
                new_population = [copy.deepcopy(population[i]) for i in sorted_indices[:elite_count]]

                # Fill remaining population with crossover and mutation
                while len(new_population) < population_size:
                    # Use epsilon-greedy selection for parent selection
                    if random.random() < current_ga_epsilon:
                        # Exploration: Random selection
                        parent1_idx = random.randrange(len(population))
                        parent2_idx = random.randrange(len(population))
                    else:
                        # Exploitation: Tournament selection
                        tournament_size = min(3, len(population))
                        parent1_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])
                        parent2_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])

                    parent1 = population[parent1_idx]
                    parent2 = population[parent2_idx]

                    # Adaptive crossover rate based on epsilon
                    effective_crossover_rate = crossover_rate * (1 - current_ga_epsilon * 0.3)

                    # Simplified crossover
                    if random.random() < effective_crossover_rate:
                        child = {}

                        # Create a random crossover point
                        tasks_ordered = sorted(parent1.keys(), key=lambda t: t.arrival_time)
                        crossover_point = random.randint(1, len(tasks_ordered) - 1)

                        # Take first part from parent1, second from parent2
                        for i, task in enumerate(tasks_ordered):
                            if i < crossover_point:
                                child[task] = parent1[task]
                            else:
                                # Make sure task exists in parent2
                                if task in parent2:
                                    child[task] = parent2[task]
                                else:
                                    child[task] = parent1[task]
                    else:
                        # No crossover, just clone parent1
                        child = copy.deepcopy(parent1)

                    # Epsilon-influenced mutation
                    # Higher mutation when epsilon is high (exploration), lower when epsilon is low (exploitation)
                    epsilon_adjusted_mutation_rate = current_mutation_rate * current_ga_epsilon + min_mutation_rate

                    # Simple mutation
                    if random.random() < epsilon_adjusted_mutation_rate:
                        # Adaptive mutation count based on epsilon
                        # More aggressive mutation when epsilon is high
                        mutation_ratio = 0.05 + current_ga_epsilon * 0.15  # 5-20% mutation
                        mutation_count = max(1, int(len(child) * mutation_ratio))
                        tasks_to_mutate = random.sample(list(child.keys()), k=mutation_count)

                        for task in tasks_to_mutate:
                            child[task] = random.choice(self.resources)

                    new_population.append(child)

                # Update population
                population = new_population

                # Calculate fitness - evaluate each solution
                fitnesses = [simplified_fitness(sol) for sol in population]

                # Check for improvement
                current_best_idx = fitnesses.index(min(fitnesses))
                current_best_fitness = fitnesses[current_best_idx]

                if current_best_fitness < best_fitness:
                    best_solution = copy.deepcopy(population[current_best_idx])
                    best_fitness = current_best_fitness
                    best_cost = simplified_fitness(best_solution)
                    logger.info(f"✓ New best solution found! Fitness: {best_fitness}, Cost: {best_cost}")
                    stagnation_counter = 0
                else:
                    stagnation_counter += 1

                # Calculate diversity (percentage of different assignments between solutions)
                diversity = _calculate_population_diversity(population)

                if diversity < diversity_threshold_low:
                    # Low diversity - increase mutation rate to encourage exploration
                    current_mutation_rate = min(max_mutation_rate, current_mutation_rate * 1.5)
                    logger.info(f"Low diversity ({diversity:.1f}%) - increasing mutation rate to {current_mutation_rate:.3f}")
                elif diversity > diversity_threshold_high:
                    # High diversity - decrease mutation rate to encourage exploitation
                    current_mutation_rate = max(min_mutation_rate, current_mutation_rate * 0.8)
                    logger.info(f"High diversity ({diversity:.1f}%) - decreasing mutation rate to {current_mutation_rate:.3f}")

                # Calculate resource distribution
                resource_distribution = _get_resource_distribution(best_solution)

                # Calculate task type distribution
                task_type_distribution = _get_task_type_distribution(best_solution)

                # Calculate task execution times
                task_times = [fast_exec_time_estimate(task, resource.id)
                            for task, resource in best_solution.items()]
                avg_time = sum(task_times) / len(task_times) if task_times else 0
                worst_time = max(task_times) if task_times else 0
                best_time = min(task_times) if task_times else 0

                # Create a hash to track solution differences
                solution_hash = hash(frozenset((task.id, resource.id)
                                            for task, resource in best_solution.items()))

                # Calculate load balance score
                load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id])
                                    for res in self.resources])

                # Write GA progress to CSV
                with open(ga_csv_path, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        gen + 1,  # Generation number
                        best_fitness,  # Best fitness
                        sum(fitnesses) / len(fitnesses),  # Average fitness
                        diversity,  # Population diversity
                        current_mutation_rate,  # Mutation rate
                        crossover_rate,  # Crossover rate
                        json.dumps(resource_distribution),  # Tasks per resource
                        load_balance,  # Load balance score
                        solution_hash  # Solution hash
                    ])

                # Record overall solution at end of each generation
                if gen % 5 == 0 or gen == generations - 1:  # Every 5 generations and the last one
                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"GA_Gen_{gen+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping at generation {gen+1} (no improvement for {max_stagnation} generations)")
                    break

            # Record final GA solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_GA_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # === STEP 5: Finalize solution and apply to resources ===
            logger.info("Phase 3/3: Finalizing solution and distributing tasks...")

            # Calculate execution time
            execution_time = time.time() - start_time
            # Calculate and log final edge-cloud distribution
            final_edge_cloud_dist = self._get_edge_cloud_distribution(best_solution)
            logger.info(f"Final Solution - Edge: {final_edge_cloud_dist['edge_percentage']:.2f}%, Cloud: {final_edge_cloud_dist['cloud_percentage']:.2f}%")

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            # Include edge-cloud distribution in summary
            logger.info(f"Edge-Cloud Distribution: {final_edge_cloud_dist['edge_percentage']:.2f}% Edge, {final_edge_cloud_dist['cloud_percentage']:.2f}% Cloud")
            # Reset resources - explicitly clear queues
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []
                logger.info(f"Reset {resource.type} queue to 0 tasks")

            # Apply solution to resources
            distributed_tasks = []
            assignment_data = []

            # Task distribution counts
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            # Temporary task assignment tracking
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process and apply all assignments, logging each step
            logger.info(f"Processing {len(best_solution)} task assignments...")
            assignment_count = 0

            # Set reference to task's assigned resource
            for task, resource in best_solution.items():
                # Verify task is valid
                if task is None:
                    logger.error("Found None task in solution!")
                    continue

                # Handle case where resource is None by assigning to least loaded resource
                if resource is None:
                    logger.warning(f"Task {task.id} has no resource assignment, assigning to least loaded resource")
                    continue

                # Set task status and add to queue
                task.status = 'CREATED'  # Ensure correct initial status
                task.assigned_resource = resource  # Maintain reference to resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1
                assignment_count += 1

                # Record assignment
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': datetime.fromtimestamp(task.arrival_time).strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type if resource else 'None',
                    'Estimated Time': fast_exec_time_estimate(task, resource.id) if resource else 'N/A'
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            logger.info(f"Completed {assignment_count} task assignments")

            # Write to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'genetic_algorithm_{timestamp}.csv')

            # Ensure the CSV is properly created and written
            try:
                with open(csv_filepath, mode='w', newline='') as f:
                    fieldnames = [
                        'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                        'Time of Arrival', 'Status', 'Assigned Node',
                        'Estimated Time'
                    ]
                    writer = csv.DictWriter(f, fieldnames=fieldnames)
                    writer.writeheader()
                    writer.writerows(assignment_data)

                logger.info(f"Successfully wrote assignment data to: {csv_filepath}")
            except Exception as e:
                logger.error(f"Error writing to CSV file: {e}")
                # Try alternate location if Google Drive isn't accessible
                alt_csv_filepath = f'genetic_algorithm_{timestamp}.csv'
                with open(alt_csv_filepath, mode='w', newline='') as f:
                    fieldnames = [
                        'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                        'Time of Arrival', 'Status', 'Assigned Node',
                        'Estimated Time'
                    ]
                    writer = csv.DictWriter(f, fieldnames=fieldnames)
                    writer.writeheader()
                    writer.writerows(assignment_data)
                logger.info(f"Wrote assignment data to alternate location: {alt_csv_filepath}")
                csv_filepath = alt_csv_filepath

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            logger.info(f"Final solution cost: {simplified_fitness(best_solution)}")
            logger.info(f"Task distribution:")
            for resource_type, count in resource_assignment_counts.items():
                if count > 0:
                    logger.info(f"  {resource_type}: {count} tasks")
            logger.info(f"Results saved to: {csv_filepath}")

            # Add detailed debug information to verify task distribution
            total_queued = 0
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]
                logger.info(f"Copied {len(resource.task_queue)} tasks to {resource.type} queue")
                queued = len(resource.task_queue)
                total_queued += queued
                logger.info(f"After distribution: {resource.type} has {queued} tasks in queue")

            logger.info(f"Total tasks queued: {total_queued} of {total_tasks} requested")

            # Return CSV paths alongside the distributed tasks
            return distributed_tasks, {
                'ga_csv': ga_csv_path,
                'solution_csv': solution_csv_path,
                'final_assignments': csv_filepath
            }

        except Exception as e:
            logger.error(f"Error in Genetic algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())

            # Just raise the exception instead of falling back to round-robin
            logger.error("Genetic algorithm failed without fallback mechanism enabled")
            raise
    def _standard_gwo_distribution(self, total_tasks: int) -> List[Task]:
        """
        Grey Wolf Optimizer for task scheduling in edge-cloud environments.

        This algorithm uses the standard GWO position update equations,
        adapted for discrete task scheduling.

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        start_time = time.time()
        logger.info(f"\n⏳ Starting Standard Grey Wolf Optimization Algorithm for {total_tasks} tasks...")

        try:
            # === STEP 1: Generate tasks and initialize task types ===
            logger.info("Phase 1/4: Task generation and initialization...")
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                } for resource in self.resources
            }

            # Create tasks with batch processing
            batch_size = 500
            initial_tasks = []

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Use random selection:
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # === STEP 2: Define optimization utility functions ===
            logger.info("Phase 2/4: Setting up optimization parameters...")

            def calculate_execution_time(task, resource_id):
                """Calculate task execution time on a given resource"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Calculate transfer time (input + output)
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']

                # Calculate processing time
                process_time = task.total_cpu_required / resource['cpu_rating']


                # Total execution time
                return transfer_time + process_time

            def calculate_objective_function(solution):
                """
                Multi-objective function based on the formula:
                F = [0.7 × Makespan + 0.3 × Utilization]

                Where:
                - Makespan: Maximum completion time across all resources
                - Utilization: Uses the U(Ri) = L(Ri) / Makespan formula
                """
                # Return high penalty for empty or invalid solutions
                if not solution:
                    return float('inf')

                # Handle unassigned tasks with high penalty
                unassigned_tasks = sum(1 for _, resource in solution.items() if not resource)
                if unassigned_tasks > 0:
                    return float('inf')

                # 1. Calculate makespan and resource processing times
                # Track completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}
                resource_processing_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                # Calculate completion times by processing tasks in order
                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        continue

                    # Calculate task execution time
                    exec_time = calculate_execution_time(task, resource.id)

                    # Task starts at either its arrival time or when resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + exec_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                    # Track total processing time for each resource (for utilization)
                    resource_processing_times[resource.id] += exec_time

                # 2. Calculate makespan (maximum completion time across all resources)
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                # 3. Calculate utilization for each resource using U(Ri) = L(Ri) / Makespan
                resource_utilizations = {}
                for resource_id, processing_time in resource_processing_times.items():
                    resource_utilizations[resource_id] = processing_time / makespan if makespan > 0 else 0

                # 4. Calculate average utilization
                avg_utilization = sum(resource_utilizations.values()) / len(self.resources) if self.resources else 0

                # 5. Apply the F = [0.7 × Makespan + 0.3 × Utilization] formula
                # Note: Since we want to maximize utilization but minimize makespan,
                # we use (1 - avg_utilization) for the utilization component
                objective_value = 0.7 * makespan + 0.3 * (1 - avg_utilization) * makespan

                return objective_value

            # === STEP 3: Initialize Grey Wolf Optimizer parameters ===
            logger.info("Phase 3/4: Initializing Grey Wolf Optimizer...")

            # GWO parameters
            num_wolves = min(30, int(total_tasks * 0.05))  # Number of search agents
            max_iterations = min(25, int(total_tasks * 0.02))  # Maximum iterations

            # Function to create a random initial solution
            def create_initial_solution():
                """Create a random initial solution for wolves"""
                solution = {}

                # Simple random assignment for each task
                for task in initial_tasks:
                    # Randomly select any resource
                    resource = random.choice(self.resources)
                    solution[task] = resource

                return solution

            # Map resources to indices for vector operations
            resource_idx_map = {resource: idx for idx, resource in enumerate(self.resources)}
            idx_resource_map = {idx: resource for idx, resource in enumerate(self.resources)}
            num_resources = len(self.resources)

            # Initialize the wolf pack with purely random solutions
            wolf_pack = []
            for i in range(num_wolves):
                solution = create_initial_solution()
                # Each wolf gets a completely random initial solution

                # Evaluate the solution
                fitness = calculate_objective_function(solution)
                wolf_pack.append({"solution": solution, "fitness": fitness})

            # Sort wolves by fitness (ascending - lower is better)
            wolf_pack.sort(key=lambda w: w["fitness"])

            # Initialize alpha, beta, and delta wolves (best three solutions)
            alpha = wolf_pack[0]
            beta = wolf_pack[1] if len(wolf_pack) > 1 else alpha
            delta = wolf_pack[2] if len(wolf_pack) > 2 else beta

            logger.info(f"Initial alpha wolf fitness: {alpha['fitness']:.2f}")

            # === STEP 4: Run Grey Wolf Optimizer with standard equations ===
            logger.info("Phase 4/4: Running Grey Wolf Optimization with standard equations...")

            for iteration in range(max_iterations):
                # Update parameter a (linearly decreases from 2 to 0)
                a = 2 - iteration * (2 / max_iterations)

                # Update each wolf's position (solution)
                for i in range(num_wolves):
                    # Get current wolf
                    wolf = wolf_pack[i]
                    current_solution = wolf["solution"]
                    new_solution = {}

                    # For each task, update its assignment using standard GWO equations
                    for task in initial_tasks:
                        # Get current resource assignment
                        current_resource = current_solution.get(task, random.choice(self.resources))
                        current_idx = resource_idx_map[current_resource]

                        # Get alpha, beta, and delta positions for this task
                        alpha_resource = alpha["solution"].get(task, random.choice(self.resources))
                        beta_resource = beta["solution"].get(task, random.choice(self.resources))
                        delta_resource = delta["solution"].get(task, random.choice(self.resources))

                        alpha_idx = resource_idx_map[alpha_resource]
                        beta_idx = resource_idx_map[beta_resource]
                        delta_idx = resource_idx_map[delta_resource]

                        # Calculate A and C vectors for alpha, beta, delta
                        A1, A2, A3 = 2 * a * random.random() - a, 2 * a * random.random() - a, 2 * a * random.random() - a
                        C1, C2, C3 = 2 * random.random(), 2 * random.random(), 2 * random.random()

                        # Calculate distance vectors (adapted for discrete indices)
                        D_alpha = abs(C1 * alpha_idx - current_idx)
                        D_beta = abs(C2 * beta_idx - current_idx)
                        D_delta = abs(C3 * delta_idx - current_idx)

                        # Calculate new position influences
                        X1 = alpha_idx - A1 * D_alpha
                        X2 = beta_idx - A2 * D_beta
                        X3 = delta_idx - A3 * D_delta

                        # Average position in continuous space
                        avg_position = (X1 + X2 + X3) / 3

                        # Convert to discrete resource index (nearest resource)
                        # Use modulo to ensure it's within valid range
                        nearest_idx = int(round(avg_position)) % num_resources
                        new_resource = idx_resource_map[nearest_idx]

                        # Apply the new assignment
                        new_solution[task] = new_resource

                    # Evaluate the new solution
                    new_fitness = calculate_objective_function(new_solution)

                    # Update the wolf if the new solution is better
                    if new_fitness < wolf["fitness"]:
                        wolf_pack[i] = {"solution": new_solution, "fitness": new_fitness}

                # Re-sort the wolf pack
                wolf_pack.sort(key=lambda w: w["fitness"])

                # Update alpha, beta, delta wolves
                new_alpha = wolf_pack[0]
                new_beta = wolf_pack[1] if len(wolf_pack) > 1 else new_alpha
                new_delta = wolf_pack[2] if len(wolf_pack) > 2 else new_beta

                # Log progress if alpha improved
                if new_alpha["fitness"] < alpha["fitness"]:
                    logger.info(f"Iteration {iteration+1}/{max_iterations}: New alpha fitness: {new_alpha['fitness']:.2f}")

                alpha, beta, delta = new_alpha, new_beta, new_delta

            # Get the best solution from GWO (alpha wolf)
            best_solution = alpha["solution"]
            best_fitness = alpha["fitness"]

            logger.info(f"GWO completed. Best fitness: {best_fitness:.2f}")

            # Finalize solution and apply to resources
            logger.info("Finalizing and distributing tasks...")

            # Calculate execution time
            execution_time = time.time() - start_time
            logger.info(f"Total algorithm execution time: {execution_time:.2f} seconds")

            # Reset resources
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []

            # Apply the final solution
            distributed_tasks = []
            assignment_data = []
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process all assignments
            for task, resource in best_solution.items():
                if task is None or resource is None:
                    continue

                # Set task status and maintain reference
                task.status = 'CREATED'
                task.assigned_resource = resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1

                # Record the assignment
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type,
                    'Estimated Time': calculate_execution_time(task, resource.id)
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            # Update resource queues
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]

            # Save assignments to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'standard_gwo_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as csv_file:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time'
                ]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Log distribution summary
            total_tasks_assigned = sum(resource_assignment_counts.values())
            logger.info("\n=== Task Distribution Summary ===")
            logger.info(f"Total tasks assigned: {total_tasks_assigned}/{total_tasks}")

            for resource_type, count in sorted(resource_assignment_counts.items()):
                percent = (count / total_tasks_assigned) * 100 if total_tasks_assigned > 0 else 0
                logger.info(f"  {resource_type}: {count} tasks ({percent:.1f}%)")

            logger.info(f"Final solution fitness: {best_fitness:.2f}")
            logger.info(f"Results saved to: {csv_filepath}")

            return distributed_tasks

        except Exception as e:
            logger.error(f"Error in Standard Grey Wolf Optimization algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())
            raise
    def _hybrid_gwo_tabu_distribution(self, total_tasks: int) -> List[Task]:
        """
        Hybrid Grey Wolf Optimizer and Tabu Search for task scheduling in edge-cloud environments.

        This algorithm combines the global exploration capabilities of Grey Wolf Optimization with
        the local exploitation strengths of Tabu Search to find an efficient task distribution.

        Modified objective function focuses solely on minimizing makespan: f(Z) = min MK(Z)

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        start_time = time.time()
        logger.info(f"\n⏳ Starting Hybrid GWO-Tabu Search Algorithm for {total_tasks} tasks...")

        try:
            # === STEP 1: Generate tasks and initialize task types ===
            logger.info("Phase 1/6: Task generation and initialization...")
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                } for resource in self.resources
            }

            # Create tasks with batch processing
            batch_size = 500
            initial_tasks = []

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    # Ensure we're using randomized task type selection
                    task_record = tasks[j]
                    # Always use random selection for task type without any patterns
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # === STEP 2: Define optimization utility functions ===
            logger.info("Phase 2/6: Setting up optimization parameters...")

            def calculate_execution_time(task, resource_id):
                """Calculate task execution time on a given resource"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Calculate transfer time (input + output)
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']

                # Calculate processing time
                process_time = task.total_cpu_required / resource['cpu_rating']


                # Total execution time
                return transfer_time + process_time

            def calculate_makespan(solution):
                """
                Calculate makespan (completion time of the last task to finish) for a solution.

                Makespan is the maximum completion time across all resources, considering
                task dependencies and resource constraints.

                Args:
                    solution: Dict mapping tasks to resources

                Returns:
                    float: Makespan value (lower is better)
                """
                # Return high penalty for invalid solutions
                if not solution:
                    return float('inf')

                # Track completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                # Process each task in order
                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        # Penalize unassigned tasks heavily
                        return float('inf')

                    # Calculate task execution time on this resource
                    exec_time = calculate_execution_time(task, resource.id)

                    # Task starts at either its arrival time or when resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + exec_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                return makespan

            # === STEP 3: Initialize Grey Wolf Optimizer parameters ===
            logger.info("Phase 3/6: Initializing Grey Wolf Optimizer...")

            # GWO parameters
            num_wolves = min(30, int(total_tasks * 0.05))  # Number of search agents
            max_iterations = min(25, int(total_tasks * 0.02))  # Maximum iterations

            # Function to create a random initial solution
            def create_initial_solution():
                """Create a completely random initial solution for wolves"""
                solution = {}

                # Simple random assignment for each task
                for task in initial_tasks:
                    # Randomly select any resource without any preferences
                    resource = random.choice(self.resources)
                    solution[task] = resource

                return solution

            # Initialize the wolf pack with purely random solutions
            wolf_pack = []
            for i in range(num_wolves):
                solution = create_initial_solution()
                # Each wolf gets a completely random solution
                # No special treatment for any wolf

                # Evaluate the solution using makespan
                fitness = calculate_makespan(solution)
                wolf_pack.append({"solution": solution, "fitness": fitness})

            # Sort wolves by fitness (ascending - lower is better)
            wolf_pack.sort(key=lambda w: w["fitness"])

            # Initialize alpha, beta, and delta wolves (best three solutions)
            alpha = wolf_pack[0]
            beta = wolf_pack[1] if len(wolf_pack) > 1 else alpha
            delta = wolf_pack[2] if len(wolf_pack) > 2 else beta

            logger.info(f"Initial alpha wolf fitness (makespan): {alpha['fitness']:.2f}")

            # === STEP 4: Run Grey Wolf Optimizer ===
            logger.info("Phase 4/6: Running Grey Wolf Optimization...")

            for iteration in range(max_iterations):
                # Update parameter a (linearly decreases from 2 to 0)
                a = 2 - iteration * (2 / max_iterations)

                # Update each wolf's position (solution)
                for i in range(num_wolves):
                    # Get current wolf
                    wolf = wolf_pack[i]
                    new_solution = {}

                    # For each task, update its assignment based on alpha, beta, delta
                    for task in initial_tasks:
                        # Calculate the influence of each leader wolf

                        # Randomly select some positions from alpha, beta, delta solutions
                        # to create diversification and influence the current wolf's solution
                        r1, r2, r3 = random.random(), random.random(), random.random()

                        # Determine which leader wolves to follow for this task
                        if r1 < 0.6:  # 60% chance to follow alpha
                            new_solution[task] = alpha["solution"].get(task, random.choice(self.resources))
                        elif r2 < 0.5:  # 30% chance to follow beta (0.6 + 0.4*0.5 = 0.8)
                            new_solution[task] = beta["solution"].get(task, random.choice(self.resources))
                        elif r3 < 0.5:  # 10% chance to follow delta (0.8 + 0.2*0.5 = 0.9)
                            new_solution[task] = delta["solution"].get(task, random.choice(self.resources))
                        else:  # 10% chance to explore randomly
                            new_solution[task] = random.choice(self.resources)

                    # Evaluate the new solution using makespan
                    new_fitness = calculate_makespan(new_solution)

                    # Update the wolf if the new solution is better
                    if new_fitness < wolf["fitness"]:
                        wolf_pack[i] = {"solution": new_solution, "fitness": new_fitness}

                # Re-sort the wolf pack
                wolf_pack.sort(key=lambda w: w["fitness"])

                # Update alpha, beta, delta wolves
                new_alpha = wolf_pack[0]
                new_beta = wolf_pack[1] if len(wolf_pack) > 1 else new_alpha
                new_delta = wolf_pack[2] if len(wolf_pack) > 2 else new_beta

                # Log progress if alpha improved
                if new_alpha["fitness"] < alpha["fitness"]:
                    logger.info(f"Iteration {iteration+1}/{max_iterations}: New alpha fitness (makespan): {new_alpha['fitness']:.2f}")

                alpha, beta, delta = new_alpha, new_beta, new_delta

            # Get the best solution from GWO (alpha wolf)
            best_gwo_solution = alpha["solution"]
            best_gwo_fitness = alpha["fitness"]

            logger.info(f"GWO completed. Best makespan: {best_gwo_fitness:.2f}")

            # === STEP 5: Enhance with Tabu Search ===
            logger.info("Phase 5/6: Enhancing solution with Tabu Search...")

            # Tabu Search parameters
            tabu_tenure = min(50, int(total_tasks * 0.03))
            tabu_iterations = min(40, int(total_tasks * 0.05))
            tabu_list = collections.deque(maxlen=tabu_tenure)

            # Start with the best GWO solution
            current_solution = copy.deepcopy(best_gwo_solution)
            current_fitness = best_gwo_fitness

            # Initialize best Tabu solution
            tabu_best_solution = copy.deepcopy(current_solution)
            tabu_best_fitness = current_fitness

            # Function to identify critical path tasks
            def identify_critical_path_tasks(solution, limit=20):
                """
                Identify tasks on the critical path - tasks that directly affect makespan.
                These are tasks that finish at or close to the overall makespan.
                """
                # Calculate resource completion times
                resource_completion_times = {resource.id: 0 for resource in self.resources}
                task_end_times = {}

                # Sort tasks by arrival time
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                # Process each task and track end times
                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        continue

                    # Calculate execution time
                    exec_time = calculate_execution_time(task, resource.id)

                    # Calculate start and end times
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    end_time = start_time + exec_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = end_time

                    # Store task end time
                    task_end_times[task] = end_time

                # Calculate overall makespan
                makespan = max(resource_completion_times.values()) if resource_completion_times else 0

                # Find tasks close to makespan
                threshold = makespan * 0.9  # Consider tasks within 10% of makespan
                critical_tasks = [task for task, end_time in task_end_times.items()
                                if end_time >= threshold]

                # If we have too few tasks, just return the tasks with highest end times
                if len(critical_tasks) < limit:
                    critical_tasks = sorted(task_end_times.keys(),
                                        key=lambda task: task_end_times[task],
                                        reverse=True)[:limit]

                return critical_tasks[:limit]  # Return at most 'limit' tasks

            # Run Tabu Search
            stagnation_counter = 0
            max_stagnation = 10

            for iteration in range(tabu_iterations):
                # Focus on critical path tasks
                task_limit = max(20, int(len(current_solution) * 0.15))
                critical_tasks = identify_critical_path_tasks(current_solution, limit=task_limit)

                # Generate and evaluate moves
                best_move = None
                best_move_fitness = float('inf')

                # For each critical task, try moving to different resources
                for task in critical_tasks:
                    current_resource = current_solution[task]

                    # Try moving to each resource
                    for resource in self.resources:
                        # Skip current assignment and tabu moves
                        if resource == current_resource or (task.id, resource.id) in tabu_list:
                            continue

                        # Make the move
                        current_solution[task] = resource

                        # Evaluate the move using makespan
                        move_fitness = calculate_makespan(current_solution)

                        # Update best move if better
                        if move_fitness < best_move_fitness:
                            best_move = (task, resource)
                            best_move_fitness = move_fitness

                        # Restore the solution
                        current_solution[task] = current_resource

                # Apply best move if found
                if best_move:
                    task, resource = best_move
                    current_solution[task] = resource
                    current_fitness = best_move_fitness

                    # Add move to tabu list
                    tabu_list.append((task.id, resource.id))

                    # Update best tabu solution if improved
                    if current_fitness < tabu_best_fitness:
                        tabu_best_solution = copy.deepcopy(current_solution)
                        tabu_best_fitness = current_fitness
                        stagnation_counter = 0
                        logger.info(f"Tabu iteration {iteration+1}/{tabu_iterations}: New best makespan: {tabu_best_fitness:.2f}")
                    else:
                        stagnation_counter += 1
                else:
                    stagnation_counter += 1

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping Tabu Search at iteration {iteration+1} (no improvement for {max_stagnation} iterations)")
                    break

            # === STEP 6: Finalize solution and apply to resources ===
            logger.info("Phase 6/6: Finalizing and distributing tasks...")

            # Compare GWO and Tabu solutions
            logger.info(f"GWO solution makespan: {best_gwo_fitness:.2f}")
            logger.info(f"Tabu enhanced solution makespan: {tabu_best_fitness:.2f}")

            # Select the better solution
            final_solution = tabu_best_solution if tabu_best_fitness < best_gwo_fitness else best_gwo_solution
            final_fitness = min(tabu_best_fitness, best_gwo_fitness)

            # Calculate execution time
            execution_time = time.time() - start_time
            logger.info(f"Total algorithm execution time: {execution_time:.2f} seconds")

            # Reset resources
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []

            # Apply the final solution
            distributed_tasks = []
            assignment_data = []
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process all assignments
            for task, resource in final_solution.items():
                if task is None or resource is None:
                    continue

                # Set task status and maintain reference
                task.status = 'CREATED'
                task.assigned_resource = resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1

                # Record the assignment
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type,
                    'Estimated Time': calculate_execution_time(task, resource.id)
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            # Update resource queues
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]

            # Save assignments to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'hybrid_gwo_tabu_makespan_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as csv_file:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time'
                ]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Log distribution summary
            total_tasks_assigned = sum(resource_assignment_counts.values())
            logger.info("\n=== Task Distribution Summary ===")
            logger.info(f"Total tasks assigned: {total_tasks_assigned}/{total_tasks}")

            for resource_type, count in sorted(resource_assignment_counts.items()):
                percent = (count / total_tasks_assigned) * 100 if total_tasks_assigned > 0 else 0
                logger.info(f"  {resource_type}: {count} tasks ({percent:.1f}%)")

            logger.info(f"Final solution makespan: {final_fitness:.2f}")
            logger.info(f"Results saved to: {csv_filepath}")

            return distributed_tasks

        except Exception as e:
            logger.error(f"Error in Hybrid GWO-Tabu Search algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())
            raise


    def _tabu_standard_gwo_distribution(self, total_tasks: int) -> List[Task]:
        """
        Tabu-GWO algorithm with standard GWO equations for task scheduling in edge-cloud environments.

        This implementation combines standard GWO position update equations for global exploration
        with Tabu Search for local exploitation to improve load balancing.

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        start_time = time.time()
        logger.info(f"\n⏳ Starting Tabu-Standard-GWO Algorithm for {total_tasks} tasks...")

        try:
            # === STEP 1: Initialize parameters and variables ===
            logger.info("Phase 1/5: Initialization of parameters and variables...")

            # Generate tasks with random arrival times
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Create tasks with random task types
            initial_tasks = []
            batch_size = 500

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Always use random selection for task type
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                } for resource in self.resources
            }

            # === STEP 2: Define utility functions ===
            logger.info("Phase 2/5: Defining utility functions...")

            def calculate_execution_time(task, resource_id):
                """Calculate task execution time on a given resource"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Calculate transfer time (input + output)
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']

                # Calculate processing time
                process_time = task.total_cpu_required / resource['cpu_rating']


                # Total execution time
                return transfer_time + process_time

            def calculate_fitness(solution):
                """
                Calculate fitness based solely on makespan minimization
                f(Z) = min MK(Z)
                """
                # Track completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                # Process each task in order
                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        return float('inf')

                    # Calculate task execution time
                    exec_time = calculate_execution_time(task, resource.id)

                    # Task starts at either its arrival time or when resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + exec_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                return makespan

            # === STEP 3: Initialize GWO parameters ===
            logger.info("Phase 3/5: Initializing Grey Wolf Optimizer with standard equations...")

            # Map resources to indices for vector operations
            resource_idx_map = {resource: idx for idx, resource in enumerate(self.resources)}
            idx_resource_map = {idx: resource for idx, resource in enumerate(self.resources)}
            num_resources = len(self.resources)

            # GWO parameters
            num_wolves = min(30, int(total_tasks * 0.05))  # Number of search agents
            max_iterations = min(20, int(total_tasks * 0.02))  # Maximum iterations

            # Initialize wolf population with random solutions
            wolf_pack = []

            for _ in range(num_wolves):
                # Create a completely random solution (position)
                solution = {}
                for task in initial_tasks:
                    solution[task] = random.choice(self.resources)

                # Calculate fitness
                fitness = calculate_fitness(solution)

                wolf_pack.append({"position": solution, "fitness": fitness})

            # Sort wolves by fitness (ascending - lower is better)
            wolf_pack.sort(key=lambda w: w["fitness"])

            # Initialize alpha, beta, delta wolves (best three positions)
            alpha = wolf_pack[0]
            beta = wolf_pack[1] if len(wolf_pack) > 1 else alpha
            delta = wolf_pack[2] if len(wolf_pack) > 2 else beta

            logger.info(f"Initial alpha wolf fitness: {alpha['fitness']:.2f}")

            # === STEP 4: Run GWO with standard position update equations ===
            logger.info("Phase 4/5: Running Grey Wolf Optimization with standard equations...")

            for iteration in range(max_iterations):
                # Update parameter a (linearly decreases from 2 to 0)
                a = 2 - iteration * (2 / max_iterations)

                # Update each wolf's position using standard GWO equations
                for i in range(num_wolves):
                    wolf = wolf_pack[i]
                    current_position = wolf["position"]
                    new_position = {}

                    # For each task, update assignment using standard GWO equations
                    for task in initial_tasks:
                        # Get current resource assignment
                        current_resource = current_position.get(task, random.choice(self.resources))
                        current_idx = resource_idx_map[current_resource]

                        # Get alpha, beta, and delta positions for this task
                        alpha_resource = alpha["position"].get(task, random.choice(self.resources))
                        beta_resource = beta["position"].get(task, random.choice(self.resources))
                        delta_resource = delta["position"].get(task, random.choice(self.resources))

                        alpha_idx = resource_idx_map[alpha_resource]
                        beta_idx = resource_idx_map[beta_resource]
                        delta_idx = resource_idx_map[delta_resource]

                        # Calculate A and C vectors for alpha, beta, delta
                        A1, A2, A3 = 2 * a * random.random() - a, 2 * a * random.random() - a, 2 * a * random.random() - a
                        C1, C2, C3 = 2 * random.random(), 2 * random.random(), 2 * random.random()

                        # Calculate distance vectors (adapted for discrete indices)
                        D_alpha = abs(C1 * alpha_idx - current_idx)
                        D_beta = abs(C2 * beta_idx - current_idx)
                        D_delta = abs(C3 * delta_idx - current_idx)

                        # Calculate new position influences
                        X1 = alpha_idx - A1 * D_alpha
                        X2 = beta_idx - A2 * D_beta
                        X3 = delta_idx - A3 * D_delta

                        # Average position in continuous space
                        avg_position = (X1 + X2 + X3) / 3

                        # Convert to discrete resource index (nearest resource)
                        # Use modulo to ensure it's within valid range
                        nearest_idx = int(round(avg_position)) % num_resources
                        new_resource = idx_resource_map[nearest_idx]

                        # Apply the new assignment
                        new_position[task] = new_resource

                    # Evaluate new position
                    new_fitness = calculate_fitness(new_position)

                    # Update the wolf if the new position is better
                    if new_fitness < wolf["fitness"]:
                        wolf_pack[i] = {"position": new_position, "fitness": new_fitness}

                # Re-sort the wolf pack
                wolf_pack.sort(key=lambda w: w["fitness"])

                # Update alpha, beta, delta wolves
                new_alpha = wolf_pack[0]
                new_beta = wolf_pack[1] if len(wolf_pack) > 1 else new_alpha
                new_delta = wolf_pack[2] if len(wolf_pack) > 2 else new_beta

                # Log progress if alpha improved
                if new_alpha["fitness"] < alpha["fitness"]:
                    logger.info(f"Iteration {iteration+1}/{max_iterations}: New alpha fitness: {new_alpha['fitness']:.2f}")

                alpha, beta, delta = new_alpha, new_beta, new_delta

            # === STEP 5: Enhance with Tabu Search ===
            logger.info("Phase 5/5: Enhancing solution with Tabu Search...")

            # Tabu Search parameters
            tabu_tenure = min(30, int(total_tasks * 0.02))  # Maximum size of Tabu List
            max_tabu_iterations = min(30, int(total_tasks * 0.03))  # Maximum iterations for Tabu Search
            tabu_list = collections.deque(maxlen=tabu_tenure)  # Use collections.deque for efficient Tabu List

            # Start with the best solution from GWO (alpha wolf's position)
            X_alpha = copy.deepcopy(alpha["position"])
            best_solution = X_alpha
            best_fitness = alpha["fitness"]

            # Global best solution for aspiration
            global_best_fitness = best_fitness

            # Tabu Search Loop with Simple Aspiration Criterion
            for m in range(max_tabu_iterations):
                # Create a temporary solution by modifying best solution
                X_temp = copy.deepcopy(best_solution)

                # Randomly select a resource to modify
                resource = random.choice(self.resources)

                # Find tasks currently not on this resource
                movable_tasks = [task for task, current_resource in X_temp.items() if current_resource != resource]

                # If no movable tasks, continue to next iteration
                if not movable_tasks:
                    continue

                # Randomly select tasks to move
                num_tasks_to_move = random.randint(1, min(5, len(movable_tasks)))
                tasks_to_move = random.sample(movable_tasks, num_tasks_to_move)

                # Move selected tasks to the new resource
                for task in tasks_to_move:
                    X_temp[task] = resource

                # Calculate fitness of the new solution
                temp_fitness = calculate_fitness(X_temp)

                # Simple Aspiration Criterion:
                # 1. If the move is not in Tabu List
                # 2. OR if the new solution is better than the global best solution
                is_tabu_move = any((task.id, resource.id) in tabu_list for task, resource in X_temp.items())

                if not is_tabu_move or temp_fitness < global_best_fitness:
                    # Update solutions if improved
                    if temp_fitness < best_fitness:
                        best_solution = X_temp
                        best_fitness = temp_fitness

                        # Update global best if significantly improved
                        if temp_fitness < global_best_fitness:
                            global_best_fitness = temp_fitness
                            logger.info(f"New global best found: {global_best_fitness:.2f}")

                    # Add move to Tabu List
                    for task in tasks_to_move:
                        tabu_list.append((task.id, resource.id))

                    logger.info(f"Tabu iteration {m+1}/{max_tabu_iterations}: Improved solution. Fitness: {best_fitness:.2f}")

            # Final solution is the best found
            final_solution = best_solution
            final_fitness = best_fitness

            ## Stop here for tabu standard implementation ###
            # Calculate execution time
            execution_time = time.time() - start_time
            logger.info(f"Total algorithm execution time: {execution_time:.2f} seconds")

            # Apply the final solution to resources
            # Rest of the code remains the same as in the original function
            # (resource resetting, task distribution, CSV saving, etc.)

            # Reset resources
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []

            # Apply the final solution
            distributed_tasks = []
            assignment_data = []
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process all assignments
            for task, resource in final_solution.items():
                if task is None or resource is None:
                    continue

                # Set task status and maintain reference
                task.status = 'CREATED'
                task.assigned_resource = resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1

                # Record the assignment
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type,
                    'Estimated Time': calculate_execution_time(task, resource.id)
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            # Update resource queues
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]

            # Save assignments to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'tabu_standard_gwo_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as csv_file:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time'
                ]
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Log distribution summary
            total_tasks_assigned = sum(resource_assignment_counts.values())
            logger.info("\n=== Task Distribution Summary ===")
            logger.info(f"Total tasks assigned: {total_tasks_assigned}/{total_tasks}")

            for resource_type, count in sorted(resource_assignment_counts.items()):
                percent = (count / total_tasks_assigned) * 100 if total_tasks_assigned > 0 else 0
                logger.info(f"  {resource_type}: {count} tasks ({percent:.1f}%)")

            logger.info(f"Final solution fitness: {final_fitness:.2f}")
            logger.info(f"Results saved to: {csv_filepath}")

            return distributed_tasks

        except Exception as e:
            logger.error(f"Error in Tabu-Standard-GWO algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())
            raise
    def _optimized_hybrid_algorithm(self, total_tasks: int) -> List[Task]:
        """
        Optimized hybrid Tabu-Genetic algorithm for smaller task sets (2000-5000 tasks).
        This version is significantly faster while maintaining solution quality.

        Key optimizations:
        1. Faster initialization with smaller population
        2. Reduced generations with early convergence detection
        3. Streamlined Tabu Search focused on high-impact tasks
        4. Simplified fitness function with multi-objective cost calculation
        5. Better progress tracking throughout execution

        Args:
            total_tasks (int): Total number of tasks to distribute

        Returns:
            List[Task]: Distributed tasks with optimized resource assignments
        """
        # Log start time and show progress indicator
        start_time = time.time()
        logger.info(f"\n⏳ Starting Optimized Hybrid Algorithm for {total_tasks} tasks...")
        logger.info("Phase 1/5: Task generation and initialization...")

        # Initialize CSV tracking for algorithm progress
        csv_folder = "/content/drive/My Drive/EdgeSimPy/algorithm_tracking"
        os.makedirs(csv_folder, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        ga_csv_path = os.path.join(csv_folder, f'ga_evolution_{timestamp}.csv')
        ts_csv_path = os.path.join(csv_folder, f'tabu_search_{timestamp}.csv')
        solution_csv_path = os.path.join(csv_folder, f'solution_comparison_{timestamp}.csv')

        # Initialize CSV files with headers
        with open(ga_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Generation', 'Best_Fitness', 'Average_Fitness', 'Population_Diversity',
                            'Mutation_Rate', 'Crossover_Rate', 'Tasks_Per_Resource',
                            'Load_Balance_Score', 'Solution_Hash'])

        with open(ts_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Iteration', 'Current_Fitness', 'Best_Fitness', 'Move_Type',
                            'Task_ID', 'Old_Resource', 'New_Resource', 'Tabu_List_Size',
                            'Improvement_Percentage', 'Solution_Hash'])

        with open(solution_csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Algorithm_Phase', 'Timestamp', 'Solution_Cost', 'Load_Balance_Score',
                            'Resource_Distribution', 'Task_Type_Distribution', 'Average_Execution_Time',
                            'Worst_Task_Time', 'Best_Task_Time'])

        try:
            # === STEP 1: Generate tasks with batch processing for efficiency ===
            tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

            # Define task types with cached requirements
            TASK_TYPES = {
                "RT1": {"input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
                "RT2": {"input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
                "RT3": {"input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
                "RT4": {"input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
                "WT1": {"input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
                "WT2": {"input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
                "WT3": {"input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
                "WT4": {"input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
            }

            # Pre-compute resource capabilities for faster lookup
            RESOURCE_CAPABILITIES = {
                resource.id: {
                    'type': resource.type,
                    'cpu_rating': resource.total_cpu_rating,
                    'bandwidth': resource.total_bandwidth,
                    'is_cloud': resource.type.startswith("Cloud_"),
                    'memory': resource.total_memory,
                } for resource in self.resources
            }

            # Helper methods for tracking metrics
            def _calculate_population_diversity(population):
                """Calculate population diversity as percentage of different assignments"""
                if len(population) <= 1:
                    return 0

                # Take the first solution as reference
                reference = population[0]
                total_assignments = len(reference) * (len(population) - 1)
                different_assignments = 0

                # Compare each solution with the reference
                for solution in population[1:]:
                    for task in reference:
                        if task in solution and reference[task] != solution[task]:
                            different_assignments += 1

                return (different_assignments / total_assignments) * 100 if total_assignments > 0 else 0

            def _get_resource_distribution(solution):
                """Get distribution of tasks per resource"""
                distribution = {resource.type: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if resource:
                        distribution[resource.type] += 1

                return distribution

            def _get_task_type_distribution(solution):
                """Get distribution of task types per resource type"""
                distribution = {}

                for task, resource in solution.items():
                    if resource:
                        resource_type = resource.type.split('_')[0]  # Cloud, Smartphone, or Raspberry
                        task_type = task.type

                        if resource_type not in distribution:
                            distribution[resource_type] = {}

                        if task_type not in distribution[resource_type]:
                            distribution[resource_type][task_type] = 0

                        distribution[resource_type][task_type] += 1

                return distribution
           # === NEW MAKESPAN WEIGHTED SUM FUNCTION ===
            def makespan_weighted_sum_fitness(solution, weights=None):
                """
                Multi-objective weighted sum fitness function focusing on:
                1. Makespan (total completion time)
                2. Average resource utilization

                Args:
                    solution: Task-to-resource mapping
                    weights: Optional weights for different objectives

                Returns:
                    float: Weighted sum fitness value (lower is better)
                """
                if weights is None:
                    # Default weights - fixed for consistent optimization
                    weights = {
                        'makespan': 0.65,
                        'resource_utilization': 0.35
                    }

                # 1. Makespan calculation
                # Track earliest completion time for each resource
                resource_completion_times = {resource.id: 0 for resource in self.resources}

                # Sort tasks by arrival time to respect task arrival order
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:
                        continue

                    # Calculate task execution time
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Task starts at either its arrival time or when the resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + total_task_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                # 2. Average resource utilization calculation
                # Track total processing time for each resource
                resource_total_processing = {resource.id: 0 for resource in self.resources}

                for task, resource in solution.items():
                    if not resource:
                        continue

                    # Calculate processing time for this task
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Add to resource's total processing time
                    resource_total_processing[resource.id] += total_task_time

                # Calculate average utilization across all resources
                if makespan > 0:
                    resource_utilizations = [processing_time / makespan for processing_time in resource_total_processing.values()]
                    avg_resource_utilization = sum(resource_utilizations) / len(self.resources)

                    # Convert to cost: higher utilization should have LOWER cost
                    utilization_cost = 1.0 / (avg_resource_utilization + 0.001)  # Add small constant to avoid division by zero
                else:
                    utilization_cost = 100000  # High penalty for zero makespan (shouldn't happen)

                # Calculate weighted sum of all objectives
                total_cost = (
                    weights['makespan'] * makespan +
                    weights['resource_utilization'] * utilization_cost
                )

                return total_cost

            # Faster task creation with larger batch size
            batch_size = 500
            initial_tasks = []
            resource_loads = {r.id: 0 for r in self.resources}

            # Show progress for task creation
            logger.info(f"Creating {total_tasks} tasks in batches of {batch_size}...")

            for i in range(0, total_tasks, batch_size):
                batch_end = min(i + batch_size, total_tasks)
                batch_tasks = []

                for j in range(i, batch_end):
                    task_record = tasks[j]
                    # Instead of cycling through task types:
                    # task_type = list(TASK_TYPES.keys())[j % len(TASK_TYPES)]

                    # Use random selection:
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    task = Task(
                        task_id=task_record.id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )
                    task.arrival_time = task_record.arrival_time
                    batch_tasks.append(task)

                initial_tasks.extend(batch_tasks)
                logger.info(f"Created {min(batch_end, total_tasks)}/{total_tasks} tasks ({(min(batch_end, total_tasks)/total_tasks)*100:.1f}%)")

            # Special handling for very small task counts (like 5)
            if total_tasks <= 10:
                logger.info(f"Small task count ({total_tasks}) detected - using specialized optimization parameters")
                # Smaller population, fewer generations for tiny problems
                population_size = max(5, min(10, total_tasks))
                generations = 3
                tabu_iterations = max(5, min(10, total_tasks))
                tabu_tenure = max(3, min(5, total_tasks))
                max_stagnation = 2
            else:
            # === STEP 2: Adjusted optimization parameters for larger task counts ===
                population_size = min(50, int(total_tasks * 0.1))  # Proportional to total tasks
                generations = min(20, int(total_tasks * 0.05))     # More generations for larger problems
                tabu_iterations = min(100, int(total_tasks * 0.05))  # Increased iterations
                tabu_tenure = min(50, int(total_tasks * 0.02))     # Slightly larger tabu tenure
                max_stagnation = 10    # Increased early stopping patience

            # === Initialize epsilon decay parameters ===
                epsilon_start = 1.0  # Start with full exploration
                epsilon_end = 0.1    # End with minimal exploration
                ga_epsilon_decay_rate = (epsilon_start - epsilon_end) / generations
                ts_epsilon_decay_rate = (epsilon_start - epsilon_end) / tabu_iterations
                current_ga_epsilon = epsilon_start
                current_ts_epsilon = epsilon_start

            # With these adaptive mutation parameters:
            base_mutation_rate = 0.15
            min_mutation_rate = 0.1
            max_mutation_rate = 0.4
            current_mutation_rate = base_mutation_rate
            diversity_threshold_low = 15  # Percentage below which to increase mutation
            diversity_threshold_high = 40  # Percentage above which to decrease mutation
            crossover_rate = 0.8
            elite_count = 2

            tabu_list = collections.deque(maxlen=tabu_tenure)

            # === STEP 3: Define optimized utility functions ===
            def fast_exec_time_estimate(task, resource_id):
                """Simplified execution time estimation for speed"""
                resource = RESOURCE_CAPABILITIES[resource_id]

                # Basic estimate combining transfer and processing time
                transfer_time = ((task.input_size + task.output_size) * 1024) / resource['bandwidth']
                process_time = task.total_cpu_required / resource['cpu_rating']


                # Simplified total time calculation
                return transfer_time + process_time

            def simplified_fitness(solution):
                """
                Multi-objective fitness function with core components only:
                1. Makespan (minimize)
                2. Throughput (maximize)
                3. Utilization (maximize)

                Fitness Formula:
                Fitness = (w_makespan * Makespan)
                        - (w_throughput * Throughput)
                        - (w_utilization * Utilization)

                Weight Control Guidelines:
                - w_makespan: Controls importance of completion time
                - Higher values (1.5-3.0): Prioritize minimizing overall completion time
                - Lower values (0.1-0.5): Reduce emphasis on completion time
                - For time-critical applications, use higher values

                - w_throughput: Controls importance of tasks processed per time unit
                - Higher values (1.5-3.0): Prioritize processing more tasks quickly
                - Lower values (0.1-0.5): Focus less on throughput, more on other objectives
                - For batch processing systems, use higher values

                - w_utilization: Controls importance of balanced resource usage
                - Higher values (1.5-3.0): Prioritize keeping resources busy
                - Lower values (0.1-0.5): Allow some resources to remain idle if it helps other objectives
                - For resource-constrained environments, use higher values

                To Control Edge-Cloud Distribution Without Explicit Penalties:
                - For more edge processing: Increase w_throughput slightly (favors lower transfer times)
                - For more cloud processing: Increase w_makespan slightly (favors faster processing)
                - A balanced approach with equal weights typically produces a natural distribution
                """
                # Weights - These can be adjusted to change optimization priorities
                w_makespan = 0.7    # Range: 0.1-3.0, Higher = prioritize minimizing completion time
                w_throughput = 1.8  # Range: 0.1-3.0, Higher = prioritize maximizing tasks per time unit
                w_utilization = 1.5 # Range: 0.1-3.0, Higher = prioritize keeping resources busy

                # Return high penalty for empty solutions
                if not solution:
                    return float('inf')

                # Calculate makespan
                resource_completion_times = {resource.id: 0 for resource in self.resources}
                sorted_tasks = sorted(solution.keys(), key=lambda task: task.arrival_time)

                for task in sorted_tasks:
                    resource = solution[task]
                    if not resource:  # Skip unassigned tasks
                        continue

                    # Calculate task execution time
                    transfer_time = ((task.input_size + task.output_size) * 1024) / resource.total_bandwidth
                    process_time = task.total_cpu_required / resource.total_cpu_rating
                    total_task_time = transfer_time + process_time

                    # Task starts at either its arrival time or when resource becomes available
                    start_time = max(task.arrival_time, resource_completion_times[resource.id])
                    completion_time = start_time + total_task_time

                    # Update resource completion time
                    resource_completion_times[resource.id] = completion_time

                # Makespan is the maximum completion time across all resources
                makespan = max(resource_completion_times.values()) if resource_completion_times else float('inf')

                # Calculate throughput (tasks/time)
                # Higher throughput is better, so we use negative coefficient
                throughput = len(solution) / makespan if makespan > 0 else 0

                # Calculate average utilization across resources WITHOUT penalties
                resource_utilizations = []
                if makespan > 0:
                    for resource in self.resources:
                        busy_time = resource_completion_times.get(resource.id, 0)
                        utilization = busy_time / makespan
                        resource_utilizations.append(utilization)

                    avg_utilization = sum(resource_utilizations) / len(resource_utilizations) if resource_utilizations else 0
                else:
                    avg_utilization = 0

                # Final fitness value (lower is better)
                # Negative coefficients for throughput and utilization since we want to maximize them
                fitness = (w_makespan * makespan) - (w_throughput * throughput) - (w_utilization * avg_utilization)

                return fitness
            # === STEP 4: Create Fully Random initial solution ===
            logger.info("Phase 2/5: Creating initial population...")

            # Create initial population
            # Create fully random initial population
            population = []
            for i in range(population_size):
                # Create completely random solution
                solution = {}
                for task in initial_tasks:
                    solution[task] = random.choice(self.resources)

                population.append(solution)

            # === STEP 5: Run Optimized Genetic Algorithm ===
            logger.info("Phase 3/5: Running genetic algorithm optimization...")

            # Evaluate initial population
            fitnesses = [simplified_fitness(sol) for sol in population]

            # Track best solution
            best_idx = fitnesses.index(min(fitnesses))
            best_solution = copy.deepcopy(population[best_idx])
            best_fitness = fitnesses[best_idx]
            best_cost = simplified_fitness(best_solution)

            # Record initial state for the solution comparison CSV
            initial_resource_distribution = json.dumps(_get_resource_distribution(best_solution))
            initial_task_type_distribution = json.dumps(_get_task_type_distribution(best_solution))
            initial_task_times = [fast_exec_time_estimate(task, resource.id) for task, resource in best_solution.items()]
            avg_time = sum(initial_task_times) / len(initial_task_times) if initial_task_times else 0
            worst_time = max(initial_task_times) if initial_task_times else 0
            best_time = min(initial_task_times) if initial_task_times else 0

            # Calculate initial load balance
            load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id]) for res in self.resources])

            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Initial_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    initial_resource_distribution,  # Resource distribution
                    initial_task_type_distribution,  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Run genetic algorithm with progress updates
            stagnation_counter = 0

            for gen in range(generations):
                # Progress indicator
                current_cost = simplified_fitness(best_solution)
                logger.info(f"Generation {gen+1}/{generations} - Current best cost: {current_cost}")

                # Update epsilon for this generation
                current_ga_epsilon = max(epsilon_end, epsilon_start - gen * ga_epsilon_decay_rate)
                logger.info(f"Current GA epsilon: {current_ga_epsilon:.3f}")

                # Sort by fitness and retain elites
                sorted_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i])
                new_population = [copy.deepcopy(population[i]) for i in sorted_indices[:elite_count]]

                # Fill remaining population with crossover and mutation
                while len(new_population) < population_size:

                    # Use epsilon-greedy selection for parent selection
                    if random.random() < current_ga_epsilon:
                        # Exploration: Random selection
                        parent1_idx = random.randrange(len(population))
                        parent2_idx = random.randrange(len(population))
                    else:
                        # Exploitation: Tournament selection
                        tournament_size = min(3, len(population))
                        parent1_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])
                        parent2_idx = min(random.sample(range(len(population)), tournament_size),
                                        key=lambda i: fitnesses[i])

                    parent1 = population[parent1_idx]
                    parent2 = population[parent2_idx]
                    # Adaptive crossover rate based on epsilon
                    effective_crossover_rate = crossover_rate * (1 - current_ga_epsilon * 0.3)

                    # Simplified crossover
                    if random.random() < effective_crossover_rate:
                        child = {}

                        # Create a random crossover point
                        tasks_ordered = sorted(parent1.keys(), key=lambda t: t.arrival_time)
                        crossover_point = random.randint(1, len(tasks_ordered) - 1)

                        # Take first part from parent1, second from parent2
                        for i, task in enumerate(tasks_ordered):
                            if i < crossover_point:
                                child[task] = parent1[task]
                            else:
                                # Make sure task exists in parent2
                                if task in parent2:
                                    child[task] = parent2[task]
                                else:
                                    child[task] = parent1[task]
                    else:
                        # No crossover, just clone parent1
                        child = copy.deepcopy(parent1)

                    # Epsilon-influenced mutation
                    # Higher mutation when epsilon is high (exploration), lower when epsilon is low (exploitation)
                    epsilon_adjusted_mutation_rate = current_mutation_rate * current_ga_epsilon + min_mutation_rate
                    # Simple mutation
                    if random.random() < epsilon_adjusted_mutation_rate:
                        # Adaptive mutation count based on epsilon
                        # More aggressive mutation when epsilon is high
                        mutation_ratio = 0.05 + current_ga_epsilon * 0.15  # 5-20% mutation
                        mutation_count = max(1, int(len(child) * mutation_ratio))
                        tasks_to_mutate = random.sample(list(child.keys()), k=mutation_count)

                        for task in tasks_to_mutate:
                            child[task] = random.choice(self.resources)

                    new_population.append(child)

                # Update population
                population = new_population

                # Calculate fitness - evaluate each solution
                fitnesses = [simplified_fitness(sol) for sol in population]

                # Check for improvement
                current_best_idx = fitnesses.index(min(fitnesses))
                current_best_fitness = fitnesses[current_best_idx]

                if current_best_fitness < best_fitness:
                    best_solution = copy.deepcopy(population[current_best_idx])
                    best_fitness = current_best_fitness
                    best_cost = simplified_fitness(best_solution)
                    logger.info(f"✓ New best solution found! Fitness: {best_fitness}, Cost: {best_cost}")
                    stagnation_counter = 0
                else:
                    stagnation_counter += 1

                # Calculate diversity (percentage of different assignments between solutions)
                diversity = _calculate_population_diversity(population)

                if diversity < diversity_threshold_low:
                    # Low diversity - increase mutation rate to encourage exploration
                    current_mutation_rate = min(max_mutation_rate, current_mutation_rate * 1.5)
                    logger.info(f"Low diversity ({diversity:.1f}%) - increasing mutation rate to {current_mutation_rate:.3f}")
                elif diversity > diversity_threshold_high:
                    # High diversity - decrease mutation rate to encourage exploitation
                    current_mutation_rate = max(min_mutation_rate, current_mutation_rate * 0.8)
                    logger.info(f"High diversity ({diversity:.1f}%) - decreasing mutation rate to {current_mutation_rate:.3f}")

                # Calculate resource distribution
                resource_distribution = _get_resource_distribution(best_solution)

                # Calculate task type distribution
                task_type_distribution = _get_task_type_distribution(best_solution)

                # Calculate task execution times
                task_times = [fast_exec_time_estimate(task, resource.id)
                            for task, resource in best_solution.items()]
                avg_time = sum(task_times) / len(task_times) if task_times else 0
                worst_time = max(task_times) if task_times else 0
                best_time = min(task_times) if task_times else 0

                # Create a hash to track solution differences
                solution_hash = hash(frozenset((task.id, resource.id)
                                            for task, resource in best_solution.items()))

                # Calculate load balance score
                load_balance = np.std([len([t for t, r in best_solution.items() if r.id == res.id])
                                    for res in self.resources])

                # Write GA progress to CSV
                with open(ga_csv_path, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        gen + 1,  # Generation number
                        best_fitness,  # Best fitness
                        sum(fitnesses) / len(fitnesses),  # Average fitness
                        diversity,  # Population diversity
                        current_mutation_rate,  # Mutation rate
                        crossover_rate,  # Crossover rate
                        json.dumps(resource_distribution),  # Tasks per resource
                        load_balance,  # Load balance score
                        solution_hash  # Solution hash
                    ])

                # Record overall solution at end of each generation
                if gen % 5 == 0 or gen == generations - 1:  # Every 5 generations and the last one
                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"GA_Gen_{gen+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= max_stagnation:
                    logger.info(f"Early stopping at generation {gen+1} (no improvement for {max_stagnation} generations)")
                    break

            # Record final GA solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_GA_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # === STEP 6: Apply focused Tabu Search to best solution ===
            logger.info("Phase 4/5: Applying focused Tabu Search with Multiple Neighborhood Structures...")

            # Define neighborhood types
            neighborhood_types = ['high_impact', 'swap_pairs', 'cluster_reassign']

            # Create a copy of the best solution
            current_solution = copy.deepcopy(best_solution)
            current_fitness = simplified_fitness(current_solution)
            tabu_best_solution = copy.deepcopy(current_solution)
            tabu_best_fitness = current_fitness

            # Function to identify high-impact tasks (focuses on tasks with largest execution times)
            def identify_high_impact_tasks(solution, limit=20):
                # Calculate execution times for each task
                task_times = []
                for task, resource in solution.items():
                    if resource:
                        exec_time = fast_exec_time_estimate(task, resource.id)
                        task_times.append((task, exec_time))

                # Sort by execution time and return top tasks
                return [task for task, _ in sorted(task_times, key=lambda x: x[1], reverse=True)[:limit]]

            # Run efficient Tabu Search
            stagnation_counter = 0

            for iteration in range(tabu_iterations):
                # Update epsilon for this iteration
                current_ts_epsilon = max(epsilon_end, epsilon_start - iteration * ts_epsilon_decay_rate)
                # Progress indicator every few iterations
                if iteration % 5 == 0 or iteration == tabu_iterations-1:
                    current_cost = simplified_fitness(tabu_best_solution)
                    logger.info(f"Tabu iteration {iteration+1}/{tabu_iterations} - Current best cost: {current_cost}, Epsilon: {current_ts_epsilon:.3f}")

                # Select neighborhood type based on iteration
                # Use epsilon to determine neighborhood size and move selection strategy
                # When epsilon is high: broader neighborhood exploration
                # When epsilon is low: more focused exploitation

                # Select neighborhood type based on epsilon and iteration
                if random.random() < current_ts_epsilon:
                    # More exploration when epsilon is high - randomize neighborhood
                    current_neighborhood = random.randint(0, len(neighborhood_types)-1)
                else:
                    # More exploitation when epsilon is low - use deterministic pattern
                    current_neighborhood = iteration % len(neighborhood_types)
                neighborhood_type = neighborhood_types[current_neighborhood]
                logger.info(f"Using neighborhood type: {neighborhood_type}")

                # Generate and evaluate a limited set of moves
                best_move = None
                best_move_fitness = float('inf')
                best_move_details = {}

                if neighborhood_type == 'high_impact':
                    # Adjust task limit based on epsilon - higher epsilon means more tasks to consider
                    task_limit_base = max(20, int(len(current_solution) * 0.2))
                    task_limit = int(task_limit_base * (1 + current_ts_epsilon))
                    high_impact_tasks = identify_high_impact_tasks(current_solution, limit=task_limit)

                    # For each high-impact task, try a few resources
                    for task in high_impact_tasks:
                        current_resource = current_solution[task]

                        # Epsilon-adjusted resource sampling
                        resource_sample_size = min(int(3 + current_ts_epsilon * 7), len(self.resources))
                        sampled_resources = random.sample(self.resources, resource_sample_size)

                        for resource in sampled_resources:
                            # Check if the move is tabu
                            is_tabu = (task.id, resource.id) in tabu_list
                            # Skip current assignment
                            if resource == current_resource:
                                continue

                            # Evaluate the move
                            current_solution[task] = resource
                            move_fitness = simplified_fitness(current_solution)

                            # *** ASPIRATION CRITERION ***
                            # Allow tabu moves if they lead to the best solution found so far
                            aspiration_satisfied = move_fitness < tabu_best_fitness
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                            if (not is_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and move_fitness < best_move_fitness:
                                best_move = (task, resource)
                                best_move_fitness = move_fitness
                                best_move_details = {
                                    'task_id': task.id,
                                    'task_type': task.type,
                                    'old_resource': current_resource.type,
                                    'new_resource': resource.type,
                                    'fitness_before': current_fitness,
                                    'fitness_after': move_fitness,
                                    'move_type': 'high_impact',
                                    'aspiration_used': is_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                                }

                                # Log if aspiration criterion was used
                                if is_tabu and aspiration_satisfied:
                                    logger.info(f"Aspiration criterion applied: Tabu move allowed for task {task.id} as it improves best solution")

                            # Restore current solution
                            current_solution[task] = current_resource

                elif neighborhood_type == 'swap_pairs':
                    # Swap pairs of tasks between resources
                    # Adjust candidate count based on epsilon
                    all_tasks = list(current_solution.keys())
                    base_candidate_count = min(40, len(all_tasks))
                    candidate_count = min(len(all_tasks), int(base_candidate_count * (1 + current_ts_epsilon)))
                    candidate_tasks = random.sample(all_tasks, candidate_count)
                    # Try swapping pairs
                    for i in range(candidate_count // 2):
                        if i * 2 + 1 >= candidate_count:
                            break

                        task1 = candidate_tasks[i * 2]
                        task2 = candidate_tasks[i * 2 + 1]

                        resource1 = current_solution[task1]
                        resource2 = current_solution[task2]

                        # Skip if tasks are on the same resource
                        if resource1 == resource2:
                            continue

                        # Check if swap is tabu (both directions)
                        swap_tabu = (task1.id, resource2.id) in tabu_list or (task2.id, resource1.id) in tabu_list

                        # Try the swap
                        current_solution[task1] = resource2
                        current_solution[task2] = resource1

                        # Evaluate
                        swap_fitness = simplified_fitness(current_solution)

                        # Aspiration criterion
                        aspiration_satisfied = swap_fitness < tabu_best_fitness

                        # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                        if (not swap_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and swap_fitness < best_move_fitness:
                            best_move = ('swap', task1, task2, resource1, resource2)
                            best_move_fitness = swap_fitness
                            best_move_details = {
                                'task1_id': task1.id,
                                'task2_id': task2.id,
                                'resource1': resource1.type,
                                'resource2': resource2.type,
                                'fitness_before': current_fitness,
                                'fitness_after': swap_fitness,
                                'move_type': 'swap_pairs',
                                'aspiration_used': swap_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                            }

                            # Log if aspiration criterion was used
                            if swap_tabu and aspiration_satisfied:
                                logger.info(f"Aspiration criterion applied: Tabu swap allowed as it improves best solution")

                        # Restore current solution
                        current_solution[task1] = resource1
                        current_solution[task2] = resource2

                elif neighborhood_type == 'cluster_reassign':
                    # Reassign clusters of related tasks
                    # Group tasks by type
                    task_clusters = {}
                    for task in current_solution:
                        if task.type not in task_clusters:
                            task_clusters[task.type] = []
                        task_clusters[task.type].append(task)

                    # For each task type, try reassigning a subset of tasks
                    for task_type, tasks in task_clusters.items():
                        # Skip very large clusters
                        if len(tasks) > 30:
                            subset_base_size = min(10, len(tasks))
                            subset_size = min(len(tasks), int(subset_base_size * (1 + current_ts_epsilon)))
                            tasks_subset = random.sample(tasks, subset_size)
                        else:
                            tasks_subset = tasks

                        # Try each target resource
                        for target_resource in self.resources:
                            # Store original assignments
                            original_assignments = {task: current_solution[task] for task in tasks_subset}
                            cluster_tabu = False

                            # Check if any reassignment would be tabu
                            for task in tasks_subset:
                                if (task.id, target_resource.id) in tabu_list:
                                    cluster_tabu = True
                                    break

                            # Reassign all tasks in subset to target resource
                            for task in tasks_subset:
                                current_solution[task] = target_resource

                            # Evaluate the cluster reassignment
                            cluster_fitness = simplified_fitness(current_solution)

                            # Aspiration criterion
                            aspiration_satisfied = cluster_fitness < tabu_best_fitness

                            # Update best move if better and either not tabu OR aspiration criterion is satisfied
                            # Update best move if better and either not tabu OR aspiration criterion is satisfied OR epsilon exploration
                            if (not cluster_tabu or aspiration_satisfied or random.random() < current_ts_epsilon * 0.3) and cluster_fitness < best_move_fitness:
                                # Deep copy the subset and assignments for later use
                                best_tasks_subset = tasks_subset.copy()
                                best_original_assignments = copy.deepcopy(original_assignments)

                                best_move = ('cluster', best_tasks_subset, target_resource, best_original_assignments)
                                best_move_fitness = cluster_fitness
                                best_move_details = {
                                    'task_type': task_type,
                                    'cluster_size': len(tasks_subset),
                                    'target_resource': target_resource.type,
                                    'fitness_before': current_fitness,
                                    'fitness_after': cluster_fitness,
                                    'move_type': 'cluster_reassign',
                                    'aspiration_used': cluster_tabu and (aspiration_satisfied or random.random() < current_ts_epsilon * 0.3)
                                }

                                # Log if aspiration criterion was used
                                if cluster_tabu and aspiration_satisfied:
                                    logger.info(f"Aspiration criterion applied: Tabu cluster move allowed as it improves best solution")

                            # Restore original assignments
                            for task, resource in original_assignments.items():
                                current_solution[task] = resource

                # Apply best move if found
                if best_move:
                    move_type = best_move_details.get('move_type', 'unknown')

                    if move_type == 'high_impact':
                        task, new_resource = best_move
                        old_resource = current_solution[task]
                        current_solution[task] = new_resource

                        # Add to tabu list
                        tabu_list.append((task.id, new_resource.id))

                        logger.info(f"Applied high-impact move: Task {task.id} moved from {old_resource.type} to {new_resource.type}")

                    elif move_type == 'swap_pairs':
                        _, task1, task2, resource1, resource2 = best_move
                        current_solution[task1] = resource2
                        current_solution[task2] = resource1

                        # Add both moves to tabu list
                        tabu_list.append((task1.id, resource2.id))
                        tabu_list.append((task2.id, resource1.id))

                        logger.info(f"Applied swap move: Tasks {task1.id} and {task2.id} swapped resources")

                    elif move_type == 'cluster_reassign':
                        _, tasks_subset, target_resource, original_assignments = best_move

                        # Apply cluster reassignment
                        for task in tasks_subset:
                            current_solution[task] = target_resource
                            # Add to tabu list
                            tabu_list.append((task.id, target_resource.id))

                        logger.info(f"Applied cluster move: {len(tasks_subset)} tasks of type {best_move_details['task_type']} moved to {target_resource.type}")

                    # Update current fitness
                    old_fitness = current_fitness
                    current_fitness = best_move_fitness

                    # Calculate improvement percentage
                    improvement_pct = (old_fitness - current_fitness) / old_fitness * 100 if old_fitness > 0 else 0

                    # Write TS progress to CSV with move type info
                    with open(ts_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            iteration + 1,  # Iteration
                            current_fitness,  # Current fitness
                            tabu_best_fitness,  # Best fitness
                            move_type,  # Move type
                            best_move_details.get('task_id', best_move_details.get('task1_id', '')),  # Task ID
                            best_move_details.get('old_resource', best_move_details.get('resource1', '')),  # Old resource
                            best_move_details.get('new_resource', best_move_details.get('resource2', '')),  # New resource
                            len(tabu_list),  # Tabu list size
                            improvement_pct,  # Improvement percentage
                            hash(frozenset((t.id, r.id) for t, r in current_solution.items()))  # Solution hash
                        ])

                    # Update best solution if improved
                    if current_fitness < tabu_best_fitness:
                        tabu_best_solution = copy.deepcopy(current_solution)
                        tabu_best_fitness = current_fitness
                        stagnation_counter = 0
                    else:
                        stagnation_counter += 1
                else:
                    stagnation_counter += 1

                    # Log iteration with no move
                    with open(ts_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            iteration + 1,  # Iteration
                            current_fitness,  # Current fitness
                            tabu_best_fitness,  # Best fitness
                            "no_move",  # Move type
                            "",  # Task ID
                            "",  # Old resource
                            "",  # New resource
                            len(tabu_list),  # Tabu list size
                            0.0,  # Improvement percentage
                            hash(frozenset((t.id, r.id) for t, r in current_solution.items()))  # Solution hash
                        ])

                # Record overall solution every few iterations
                if iteration % 10 == 0 or iteration == tabu_iterations - 1:
                    # Calculate metrics similar to GA section
                    resource_distribution = _get_resource_distribution(tabu_best_solution)
                    task_type_distribution = _get_task_type_distribution(tabu_best_solution)
                    task_times = [fast_exec_time_estimate(task, resource.id)
                                for task, resource in tabu_best_solution.items()]
                    avg_time = sum(task_times) / len(task_times) if task_times else 0
                    worst_time = max(task_times) if task_times else 0
                    best_time = min(task_times) if task_times else 0
                    load_balance = np.std([len([t for t, r in tabu_best_solution.items() if r.id == res.id])
                                        for res in self.resources])

                    with open(solution_csv_path, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            f"TS_Iter_{iteration+1}",  # Algorithm phase
                            datetime.now().strftime("%H:%M:%S"),  # Timestamp
                            tabu_best_fitness,  # Solution cost
                            load_balance,  # Load balance score
                            json.dumps(resource_distribution),  # Resource distribution
                            json.dumps(task_type_distribution),  # Task type distribution
                            avg_time,  # Average execution time
                            worst_time,  # Worst task time
                            best_time  # Best task time
                        ])

                # Early stopping
                if stagnation_counter >= 5:
                    logger.info(f"Early stopping Tabu Search at iteration {iteration+1} (no improvement for 5 iterations)")
                    break

            # Record final TS solution
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_TS_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    tabu_best_fitness,  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(_get_resource_distribution(tabu_best_solution)),  # Resource distribution
                    json.dumps(_get_task_type_distribution(tabu_best_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])
            # Add edge-cloud distribution logging here
            tabu_edge_cloud_dist = self._get_edge_cloud_distribution(tabu_best_solution)
            logger.info(f"Tabu Solution - Edge: {tabu_edge_cloud_dist['edge_percentage']:.2f}%, Cloud: {tabu_edge_cloud_dist['cloud_percentage']:.2f}%")

            # === STEP 7: Select final solution and apply to resources ===
            logger.info("Phase 5/5: Finalizing solution and distributing tasks...")

            # Validate solution before proceeding
            valid_assignments = 0
            for task, resource in tabu_best_solution.items():
                if resource is not None:
                    valid_assignments += 1

            logger.info(f"Validating tabu solution: {valid_assignments} valid assignments out of {len(tabu_best_solution)} tasks")

            # Check GA solution as well
            ga_valid_assignments = 0
            for task, resource in best_solution.items():
                if resource is not None:
                    ga_valid_assignments += 1

            logger.info(f"Validating GA solution: {ga_valid_assignments} valid assignments out of {len(best_solution)} tasks")
            tabu_cost = simplified_fitness(tabu_best_solution)

            final_solution = tabu_best_solution
            logger.info(f"Using Tabu-improved solution (cost: {tabu_cost}, valid assignments: {valid_assignments})")

            # Calculate execution time
            execution_time = time.time() - start_time
            # Calculate and log final edge-cloud distribution
            final_edge_cloud_dist = self._get_edge_cloud_distribution(final_solution)
            logger.info(f"Final Solution - Edge: {final_edge_cloud_dist['edge_percentage']:.2f}%, Cloud: {final_edge_cloud_dist['cloud_percentage']:.2f}%")

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            # Include edge-cloud distribution in summary
            logger.info(f"Edge-Cloud Distribution: {final_edge_cloud_dist['edge_percentage']:.2f}% Edge, {final_edge_cloud_dist['cloud_percentage']:.2f}% Cloud")
            # Reset resources - explicitly clear queues
            for resource in self.resources:
                resource.task_queue = []
                resource.failed_tasks = []
                logger.info(f"Reset {resource.type} queue to 0 tasks")

            # Apply solution to resources
            distributed_tasks = []
            assignment_data = []
            base_time = datetime.now()

            # Task distribution counts
            resource_assignment_counts = {r.type: 0 for r in self.resources}
            # Temporary task assignment tracking
            temp_resource_task_queues = {resource.id: [] for resource in self.resources}

            # Process and apply all assignments, logging each step
            logger.info(f"Processing {len(final_solution)} task assignments...")
            assignment_count = 0

            # CRITICALLY IMPORTANT: Set reference to task's assigned resource
            # This helps maintain the connection between tasks and resources
            for task, resource in final_solution.items():
                # Verify task is valid
                if task is None:
                    logger.error("Found None task in solution!")
                    continue

                # Handle case where resource is None by assigning to least loaded resource
                if resource is None:
                    logger.warning(f"Task {task.id} has no resource assignment, assigning to least loaded resource")
                    continue

                # Set task status and add to queue
                task.status = 'CREATED'  # Ensure correct initial status
                task.assigned_resource = resource  # CRITICAL: Maintain reference to resource
                temp_resource_task_queues[resource.id].append(task)
                resource_assignment_counts[resource.type] += 1
                assignment_count += 1

                # Debug log for each assignment
                logger.info(f"Assigned Task {task.id} ({task.type}) to {resource.type}, queue now has {len(resource.task_queue)} tasks")

                # Record assignment
                # Record assignment using direct timestamp conversion without offset
                arrival_time = datetime.fromtimestamp(task.arrival_time)
                record = {
                    'Task ID': task.id,
                    'Type': task.type,
                    'Input Size (GB)': task.input_size,
                    'Output Size (GB)': task.output_size,
                    'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'Status': task.status,
                    'Assigned Node': resource.type if resource else 'None',
                    'Estimated Time': fast_exec_time_estimate(task, resource.id) if resource else 'N/A'
                }
                assignment_data.append(record)
                distributed_tasks.append(task)

            logger.info(f"Completed {assignment_count} task assignments")

            # Write to CSV
            csv_folder = "/content/drive/My Drive/CSV_dump"
            os.makedirs(csv_folder, exist_ok=True)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            csv_filepath = os.path.join(csv_folder, f'optimized_hybrid_{timestamp}.csv')

            with open(csv_filepath, mode='w', newline='') as f:
                fieldnames = [
                    'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
                    'Time of Arrival', 'Status', 'Assigned Node',
                    'Estimated Time'
                ]
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(assignment_data)

            # Print summary
            logger.info("\n=== Optimization Complete ===")
            logger.info(f"Total execution time: {execution_time:.2f} seconds")
            logger.info(f"Final solution cost: {simplified_fitness(final_solution)}")
            logger.info(f"Task distribution:")
            for resource_type, count in resource_assignment_counts.items():
                if count > 0:
                    logger.info(f"  {resource_type}: {count} tasks")
            logger.info(f"Results saved to: {csv_filepath}")

            # Record final comparison
            with open(solution_csv_path, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    "Final_Applied_Solution",  # Algorithm phase
                    datetime.now().strftime("%H:%M:%S"),  # Timestamp
                    simplified_fitness(final_solution),  # Solution cost
                    load_balance,  # Load balance score
                    json.dumps(resource_assignment_counts),  # Resource distribution (actual applied counts)
                    json.dumps(_get_task_type_distribution(final_solution)),  # Task type distribution
                    avg_time,  # Average execution time
                    worst_time,  # Worst task time
                    best_time  # Best task time
                ])

            # Add detailed debug information to verify task distribution
            total_queued = 0
            for resource in self.resources:
                resource.task_queue = temp_resource_task_queues[resource.id]
                logger.info(f"Copied {len(resource.task_queue)} tasks to {resource.type} queue")
                queued = len(resource.task_queue)
                total_queued += queued
                logger.info(f"After distribution: {resource.type} has {queued} tasks in queue")

            logger.info(f"Total tasks queued: {total_queued} of {total_tasks} requested")

            # Ensure we always have the right number of tasks
            # If we don't have enough tasks, apply Tabu-based logic for additional tasks
            if total_queued < total_tasks:
                logger.warning(f"Distribution incomplete: only {total_queued} of {total_tasks} tasks distributed")

                # Calculate how many more tasks we need
                tasks_needed = total_tasks - total_queued
                logger.info(f"Adding {tasks_needed} additional tasks using Tabu optimization logic")

                # Create additional tasks
                additional_tasks = []
                for i in range(tasks_needed):
                    # Get task type
                    # task_type = list(TASK_TYPES.keys())[i % len(TASK_TYPES)]
                    task_type = random.choice(list(TASK_TYPES.keys()))
                    specs = TASK_TYPES[task_type]

                    # Create task
                    new_task_id = total_queued + i + 1
                    task = Task(
                        task_id=new_task_id,
                        task_type=task_type,
                        input_size=specs["input_size"],
                        output_size=specs["output_size"],
                        cpu_required=specs["cpu_required"]
                    )

                    # Set arrival time
                    task.arrival_time = datetime.now().timestamp()
                    task.status = 'CREATED'
                    additional_tasks.append(task)

                # Apply Tabu-based assignment logic for the additional tasks
                # First calculate current resource loads from existing assignments
                current_loads = {r.id: len(r.task_queue) for r in self.resources}

                # For each additional task, assign based on the same criteria used in Tabu Search
                for task in additional_tasks:
                    # Find best resource based on type compatibility and load
                    best_resource = None
                    best_cost = float('inf')

                    for resource in self.resources:
                        # Calculate cost based on the same logic used in Tabu Search
                        exec_time = fast_exec_time_estimate(task, resource.id)

                        # With this (remove the conditional completely):
                        # No task-type specific penalties
                        # Add load balancing factor
                        load_factor = current_loads[resource.id] * 500
                        total_cost = exec_time + load_factor

                        if total_cost < best_cost:
                            best_cost = total_cost
                            best_resource = resource

                    # Assign to the best resource
                    if best_resource:
                        task.assigned_resource = best_resource
                        best_resource.task_queue.append(task)
                        current_loads[best_resource.id] += 1
                        distributed_tasks.append(task)
                        logger.info(f"Added task {task.id} ({task.type}) to {best_resource.type} using Tabu logic")

                # Verify final count
                final_count = sum(len(r.task_queue) for r in self.resources)
                logger.info(f"Final distribution check: {final_count} of {total_tasks} tasks distributed")

            logger.info(f"Returning {len(distributed_tasks)} distributed tasks")

            # Return CSV paths alongside the distributed tasks
            return distributed_tasks, {
                'ga_csv': ga_csv_path,
                'ts_csv': ts_csv_path,
                'solution_csv': solution_csv_path,
                'final_assignments': csv_filepath
            }

        except Exception as e:
            logger.error(f"Error in Optimized Hybrid algorithm: {e}")
            import traceback
            logger.error(traceback.format_exc())

            # Just raise the exception instead of falling back to round-robin
            logger.error("Hybrid algorithm failed without fallback mechanism enabled")
            raise

    def _shortest_job_first_distribution(self, total_tasks: int) -> List[Task]:
        """
        Distribute tasks using Shortest Job First approach. Tasks are sorted by estimated
        execution time before distribution, while maintaining original task validation logic.
        """
        # Step 1: Generate tasks and arrival times
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Create temporary tasks with execution time estimates
        task_estimates = []
        for task_record in tasks:
            #task_type = task_types[task_record.id % len(task_types)]
            task_type = random.choice(task_types)

            # Create temporary task for estimation
            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            # Find minimum execution time across all compatible resources
            min_execution_time = float('inf')
            for resource in self.resources:
                can_process, _ = resource.can_process_task(temp_task)
                if can_process:
                    estimated_time = temp_task.estimate_execution_time(resource)
                    min_execution_time = min(min_execution_time, estimated_time)

            task_estimates.append({
                'task_record': task_record,
                'task_type': task_type,
                'estimated_time': min_execution_time
            })

        # Step 3: Sort tasks by estimated execution time
        sorted_tasks = sorted(task_estimates, key=lambda x: x['estimated_time'])

        # Step 4: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 5: Pre-validate tasks (keeping original validation logic)
        for task_info in sorted_tasks:
            task_type = task_info['task_type']
            task_record = task_info['task_record']

            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 6: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 7: Distribute sorted tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_info in sorted_tasks:
            task_record = task_info['task_record']
            task_type = task_info['task_type']
            resource = self.resources[resource_index]

            # Create the actual task
            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time

            arrival_time = base_time + timedelta(seconds=task_record.arrival_time)
            assigned_node = resource.type

            # Check if resource can process this task
            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            # Record task assignment
            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_type["input_size"],
                'Output Size': task_type["output_size"],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node,
                'Estimated Time': task_info['estimated_time']
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)


        # Logging and verification steps
        logger.info("\n=== Task Distribution Summary (SJF) ===")
        logger.info(f"Total tasks requested: {total_tasks}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_sjf_csv(csv_data)

        return distributed_tasks
    def _round_robin_distribution(self, total_tasks: int) -> List[Task]:
        """
        Distribute tasks using Round Robin approach
        """
        # Step 1: Generate tasks and arrival times
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Step 1: Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 3: Pre-validate tasks
        for task_record in tasks:
            # task_type = task_types[task_record.id % len(task_types)]
            task_type = random.choice(task_types)
            temp_task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )

            # Create temporary task for estimation
            temp_task = Task(
                task_id=task_record.id,
                task_type=random.choice(task_types)["type"],  # Randomly select task type
                input_size=random.choice(task_types)["input_size"],
                output_size=random.choice(task_types)["output_size"],
                cpu_required=random.choice(task_types)["cpu_required"]
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 4: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 5: Distribute tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_record in tasks:
            resource = self.resources[resource_index]

            # Determine task type
            # task_type = task_types[task_record.id % len(task_types)]
            task_type = random.choice(task_types)

            task = Task(
                task_id=task_record.id,
                task_type=task_type["type"],
                input_size=task_type["input_size"],
                output_size=task_type["output_size"],
                cpu_required=task_type["cpu_required"]
            )
            task.arrival_time = task_record.arrival_time

            arrival_time = base_time + timedelta(seconds=task_record.arrival_time)
            assigned_node = resource.type

            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_type["input_size"],
                'Output Size': task_type["output_size"],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)

        # Logging and verification steps
        logger.info("\n=== Task Distribution Summary ===")
        logger.info(f"Total tasks requested: {total_tasks}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_csv(csv_data)

        return distributed_tasks
    def _Default_distribute_tasks(self, total_tasks: int) -> List[Task]:
        """
        Guaranteed distribution of exactly total_tasks tasks.
        Uses pre-validation, strict counting, and saves distribution data to CSV.
        """
        tasks, cumulative_times, inter_arrival_times = self.generate_tasks(total_tasks)

        # Step 1: Define task types with input/output sizes
        task_types = [
            # Read Tasks: input_size > 0, output_size = 0
            {"type": "RT1", "input_size": 5.0, "output_size": 0, "cpu_required": 2_000_000},
            {"type": "RT2", "input_size": 0.2, "output_size": 0, "cpu_required": 4_000_000},
            {"type": "RT3", "input_size": 5.0, "output_size": 0, "cpu_required": 200_000},
            {"type": "RT4", "input_size": 0.5, "output_size": 0, "cpu_required": 500_000},
            # Write Tasks: input_size = 0, output_size > 0
            {"type": "WT1", "input_size": 0, "output_size": 2.0, "cpu_required": 2_000_000},
            {"type": "WT2", "input_size": 0, "output_size": 0.5, "cpu_required": 1_000_000},
            {"type": "WT3", "input_size": 0, "output_size": 5.0, "cpu_required": 500_000},
            {"type": "WT4", "input_size": 0, "output_size": 0.2, "cpu_required": 200_000}
        ]

        # Step 2: Generate task records
        task_records = []
        for i, arrival_time in enumerate(cumulative_times):
            task_type = task_types[i % len(task_types)]
            task_records.append({
                'id': i + 1,
                'type': task_type["type"],
                'input_size': task_type["input_size"],
                'output_size': task_type["output_size"],
                'cpu_required': task_type["cpu_required"],
                'arrival_time': arrival_time
            })

        # Step 3: Create resource task map
        resource_task_map = {resource.id: {'can_process': [], 'cannot_process': []}
                            for resource in self.resources}

        # Step 4: Pre-validate tasks
        for task_record in task_records:
            temp_task = Task(
                task_id=task_record['id'],
                task_type=task_record['type'],
                input_size=task_record['input_size'],
                output_size=task_record['output_size'],
                cpu_required=task_record['cpu_required']
            )

            for resource in self.resources:
                can_process, reason = resource.can_process_task(temp_task)
                if can_process:
                    resource_task_map[resource.id]['can_process'].append(task_record)
                else:
                    resource_task_map[resource.id]['cannot_process'].append(task_record)

        # Step 5: Reset resources
        for resource in self.resources:
            resource.task_queue = []
            resource.failed_tasks = []

        # Step 6: Distribute tasks
        distributed_tasks = []
        resource_index = 0
        csv_data = []
        base_time = datetime.now()

        for task_record in task_records:
            resource = self.resources[resource_index]

            task = Task(
                task_id=task_record['id'],
                task_type=task_record['type'],
                input_size=task_record['input_size'],
                output_size=task_record['output_size'],
                cpu_required=task_record['cpu_required']
            )
            task.arrival_time = task_record['arrival_time']

            arrival_time = base_time + timedelta(seconds=task_record['arrival_time'])
            assigned_node = resource.type

            if task_record in resource_task_map[resource.id]['can_process']:
                task.status = 'READY'  # Changed from 'queued' to match Task class states
                resource.task_queue.append(task)
            else:
                task.status = 'FAILED'  # Changed from 'failed' to match Task class states
                task.failure_reason = f"Cannot process on {resource.type}"
                resource.failed_tasks.append(task)

            csv_data.append({
                'Task ID': task.id,
                'Type': task.type,
                'Input Size': task_record['input_size'],
                'Output Size': task_record['output_size'],
                'Time of Arrival': arrival_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Status': task.status,
                'Assigned Node': assigned_node
            })

            distributed_tasks.append(task)
            resource_index = (resource_index + 1) % len(self.resources)

        # Step 7: Verify counts
        distributed_count = len(distributed_tasks)
        queued_count = sum(len(r.task_queue) for r in self.resources)
        failed_count = sum(len(r.failed_tasks) for r in self.resources)
        total_count = queued_count + failed_count

        # Log summary
        logger.info("\n=== Task Distribution Summary ===")
        logger.info(f"Total tasks requested: {total_tasks}")
        logger.info(f"Tasks distributed: {distributed_count}")
        logger.info(f"Tasks queued: {queued_count}")
        logger.info(f"Tasks failed: {failed_count}")
        logger.info(f"Total count: {total_count}")

        # Resource breakdown
        for resource in self.resources:
            queued = len(resource.task_queue)
            failed = len(resource.failed_tasks)
            logger.info(
                f"{resource.type}: "
                f"Queue={queued}, Failed={failed}, "
                f"Total={queued + failed}"
            )

        # Verify counts
        assert distributed_count == total_tasks, \
            f"Distribution count mismatch: {distributed_count} != {total_tasks}"
        assert total_count == total_tasks, \
            f"Total count mismatch: {total_count} != {total_tasks}"
        assert len(set(t.id for t in distributed_tasks)) == total_tasks, \
            f"Task ID uniqueness violation"

        # Write data to CSV
        self.write_tasks_to_csv(csv_data)

        return distributed_tasks

    def write_tasks_to_sjf_csv(self, task_data):
        """
        Write task distribution data to CSV with updated filename for SJF algorithm
        """
        # Create CSV folder if it doesn't exist
        csv_folder = "/content/drive/My Drive/CSV_dump"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        # Generate filename with timestamp and 'sjf' identifier
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'sjf_task_distribution_{timestamp}.csv')

        # Define CSV fields
        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Start Time', 'Input Transfer Time',
            'Processing Time', 'Output Transfer Time', 'Total Time',
            'Status', 'Assigned Node', 'Estimated Time'
        ]

        # Write data to CSV
        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for task in task_data:
                writer.writerow({
                    'Task ID': task['Task ID'],
                    'Type': task['Type'],
                    'Input Size (GB)': task.get('Input Size', 0),
                    'Output Size (GB)': task.get('Output Size', 0),
                    'Time of Arrival': task['Time of Arrival'],
                    'Start Time': task.get('Start Time', ''),
                    'Input Transfer Time': task.get('Input Transfer Time', ''),
                    'Processing Time': task.get('Processing Time', ''),
                    'Output Transfer Time': task.get('Output Transfer Time', ''),
                    'Total Time': task.get('Total Time', ''),
                    'Status': task['Status'],
                    'Assigned Node': task['Assigned Node'],
                    'Estimated Time': task.get('Estimated Time', 'N/A')
                })

        logger.info(f"Task distribution data written to {csv_filepath}")
        return csv_filepath

    def write_tasks_to_csv(self, task_data):
        """
        Updated CSV output to include transfer phases and data sizes
        """
        csv_folder = "/content/drive/My Drive/CSV_dump"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'round_robin_distribution_{timestamp}.csv')

        # Updated fieldnames to include transfer information
        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Start Time', 'Input Transfer Time',
            'Processing Time', 'Output Transfer Time', 'Total Time',
            'Status', 'Assigned Node'
        ]

        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            for task in task_data:
                writer.writerow({
                    'Task ID': task['Task ID'],
                    'Type': task['Type'],
                    'Input Size (GB)': task.get('input_size', 0),
                    'Output Size (GB)': task.get('output_size', 0),
                    'Time of Arrival': task['Time of Arrival'],
                    'Start Time': task.get('start_time', ''),
                    'Input Transfer Time': task.get('input_transfer_time', ''),
                    'Processing Time': task.get('processing_time', ''),
                    'Output Transfer Time': task.get('output_transfer_time', ''),
                    'Total Time': task.get('total_time', ''),
                    'Status': task['Status'],
                    'Assigned Node': task['Assigned Node']
                })

        logger.info(f"Task distribution data written to {csv_filepath}")

        # Flush and sync to ensure all data is written to the drive
        logger.info("Google Drive has been unmounted. You can now access the CSV file in your Google Drive.")
    def distribute_tasks(self, total_tasks: int, distribution_type: str = 'default'):
        """
        Enhanced distribution method with verification and emergency fallback
        """
        # Mapping of distribution strategies
        distribution_strategies = {
            'default': self._Default_distribute_tasks,
            'round_robin': self._round_robin_distribution,
            'shortest_job_first': self._shortest_job_first_distribution,
            'hybrid': self._optimized_hybrid_algorithm,
            'greywolf': self._standard_gwo_distribution,
            'genetic_modified': self.genetic_modified,
            'gwo_tabu': self._hybrid_gwo_tabu_distribution,
            'tabu_gwo': self._tabu_standard_gwo_distribution,

        }

        # Select and execute the appropriate distribution strategy
        if distribution_type in distribution_strategies:
            try:
                logger.info(f"Using {distribution_type} distribution strategy for {total_tasks} tasks")

                # First attempt with selected algorithm
                result = distribution_strategies[distribution_type](total_tasks)

                # Handle different return types
                if isinstance(result, tuple) and len(result) == 2:
                    distributed_tasks, csv_paths = result
                else:
                    distributed_tasks = result

                # Verify tasks were actually queued and fix if needed
                total_queued = sum(len(resource.task_queue) for resource in self.resources)
                if total_queued < total_tasks:
                    logger.warning(f"Verification failed: Only {total_queued} of {total_tasks} queued after {distribution_type}")
                    fixed_tasks = self.verify_and_fix_task_distribution(total_tasks)

                    # If we got CSV paths, package them with the fixed tasks
                    if 'csv_paths' in locals():
                        return fixed_tasks, csv_paths
                    else:
                        return fixed_tasks

                # Return the original result
                return result

            except Exception as e:
                logger.error(f"Error in {distribution_type} distribution: {e}")
                logger.info("Falling back to emergency distribution due to error")
                return self.verify_and_fix_task_distribution(total_tasks)
        else:
            raise ValueError(f"Unsupported distribution type: {distribution_type}. "
                            f"Supported types are: {list(distribution_strategies.keys())}")
    def _write_tabu_assignments_to_csv(self, assignment_data):
        """Helper method to write Tabu Search assignments to CSV with total cost."""
        csv_folder = "/content/drive/My Drive/CSV_dump"
        os.makedirs(csv_folder, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'tabu_task_distribution_{timestamp}.csv')

        # Calculate total cost for each task when written
        for task_record in assignment_data:
            # Skip if no resource assigned
            if task_record['Assigned Node'] == 'None':
                task_record['Total Cost'] = 'N/A'
                continue

            # Base execution time
            base_cost = float(task_record['Estimated Time'])
            total_cost = base_cost

            # Add WT3-Raspberry penalty if applicable
            if task_record['Type'] == 'WT3' and task_record['Assigned Node'].startswith('Raspberry_'):
                total_cost += base_cost * 3  # 5x penalty

            # Add cloud penalty if applicable
            elif task_record['Assigned Node'].startswith('Cloud_') and task_record['Type'] not in ['RT1', 'RT3']:
                total_cost += base_cost * 0.5

            task_record['Total Cost'] = f"{total_cost:.2f}"

        fieldnames = [
            'Task ID', 'Type', 'Input Size (GB)', 'Output Size (GB)',
            'Time of Arrival', 'Status', 'Assigned Node', 'Estimated Time',
            'Total Cost'  # Added Total Cost field
        ]

        with open(csv_filepath, mode='w', newline='') as csv_file:
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(assignment_data)

        logger.info(f"Task assignments with costs written to {csv_filepath}")
        return csv_filepath

    def generate_live_feed(self):
        """
        Updated live visualization to show transfer phases
        """
        table = Table(
            title="Live Resource Processing Status",
            box=box.MINIMAL_DOUBLE_HEAD,
            show_header=True,
            header_style="bold magenta",
            border_style="bold green",
            show_lines=True,
            padding=(0, 1)
        )

        # Updated columns to show transfer phases
        table.add_column("Resource", style="bold blue", width=15)
        table.add_column("Type", style="bold cyan", width=20)
        table.add_column("Current Task", style="bold yellow", width=15)
        table.add_column("Phase", style="bold green", width=26)
        table.add_column("Progress", style="bold red", width=18)
        table.add_column("Data Size", style="bold yellow", width=18)
        table.add_column("Queue", style="bold green", width=15)
        table.add_column("Completed", style="bold magenta", width=15)
        table.add_column("Failed", style="bold red", width=15)

        for resource in self.resources:
            current_task = "None"
            phase = "Idle"
            progress = "N/A"
            data_size = "N/A"

            if resource.current_task:
                task = resource.current_task
                current_task = f"{task.id} ({task.type})"

                # Phase and progress based on task status
                phase_colors = {
                    'TRANSFERRING_INPUT': "yellow",
                    'PROCESSING': "blue",
                    'TRANSFERRING_OUTPUT': "cyan",
                    'COMPLETED': "green"
                }

                phase = task.status
                progress = f"{task.completion_percentage:.1f}%"

                # Show relevant data size based on phase
                if task.status == 'TRANSFERRING_INPUT':
                    data_size = f"↑{task.input_size:.1f}GB"
                elif task.status == 'TRANSFERRING_OUTPUT':
                    data_size = f"↓{task.output_size:.1f}GB"
                else:
                    data_size = f"↕{max(task.input_size, task.output_size):.1f}GB"

            # Add row with updated information
            table.add_row(
                f"{resource.id:<8}",
                f"{resource.type:<11}",
                current_task,
                phase,
                progress,
                data_size,
                f"{len(resource.task_queue):<5}",
                f"{len(resource.completed_tasks):<9}",
                str(len(resource.failed_tasks))
            )

        return table
    def write_simulation_results(self, metrics, start_time, distribution_type):
        """
        Write simulation results to CSV file with enhanced edge-cloud distribution metrics

        Args:
            metrics (dict): Simulation metrics
            start_time (float): Simulation start timestamp
            distribution_type (str): Type of distribution algorithm used
        """
        csv_folder = "/content/drive/My Drive/EdgeSimPy/results"
        if not os.path.exists(csv_folder):
            os.makedirs(csv_folder)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        csv_filepath = os.path.join(csv_folder, f'run_simulation_results_{distribution_type}_{timestamp}.csv')

        # Prepare simulation results
        results = {
            'Distribution_Algorithm': distribution_type,
            'Simulation_Start_Time': datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S'),
            'Simulation_End_Time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Total_Tasks': metrics['total_tasks'],
            'Completed_Tasks': metrics['completed_tasks'],
            'Failed_Tasks': metrics['failed_tasks'],
            'Makespan': f"{metrics['makespan']:.2f}s",
            'Throughput': f"{metrics['throughput']:.2f} tasks/s",
        }

        # Add edge-cloud distribution metrics
        if 'edge_cloud_distribution' in metrics:
            edge_cloud = metrics['edge_cloud_distribution']
            results.update({
                'Edge_Task_Percentage': f"{edge_cloud['edge_percentage']:.2f}%",
                'Cloud_Task_Percentage': f"{edge_cloud['cloud_percentage']:.2f}%",
                'Edge_Task_Count': edge_cloud['edge_tasks'],
                'Cloud_Task_Count': edge_cloud['cloud_tasks'],
            })

        # Add load balancing metrics if available
        if 'load_balancing_metrics' in metrics:
            lb_metrics = metrics['load_balancing_metrics']
            results.update({
                'Average_Resource_Utilization': f"{lb_metrics.get('average_utilization', 0):.4f}",
                'Coefficient_of_Variation': f"{lb_metrics.get('load_balance_score', 0):.4f}",
                'CoV_Standard_Deviation': f"{lb_metrics.get('utilization_std_dev', 0):.4f}"
            })

            # Add resource type specific metrics
            if 'resource_type_metrics' in lb_metrics:
                for resource_type, type_metrics in lb_metrics['resource_type_metrics'].items():
                    if resource_type != 'inter_type_balance':
                        results.update({
                            f'{resource_type}_avg_utilization': f"{type_metrics.get('average_utilization', 0):.4f}",
                            f'{resource_type}_CoV': f"{type_metrics.get('load_balance_score', 0):.4f}"
                        })

                # Add inter-type balance if available
                if 'inter_type_balance' in lb_metrics['resource_type_metrics']:
                    inter_balance = lb_metrics['resource_type_metrics']['inter_type_balance']
                    results.update({
                        'Inter_Type_CoV': f"{inter_balance.get('balance_score', 0):.4f}"
                    })

        # Add timing metrics if available
        if 'average_turnaround_time' in metrics:
            results['Average_Turnaround_Time'] = f"{metrics['average_turnaround_time']:.2f}s"
        if 'average_waiting_time' in metrics:
            results['Average_Waiting_Time'] = f"{metrics['average_waiting_time']:.2f}s"
        if 'average_execution_time' in metrics:
            results['Average_Execution_Time'] = f"{metrics['average_execution_time']:.2f}s"

        # Add failed tasks breakdown
        for task_type, count in metrics['failed_tasks_by_type'].items():
            results[f'Failed_{task_type}'] = count

        for resource_type, count in metrics['failed_tasks_by_resource'].items():
            results[f'Failed_On_{resource_type}'] = count

        # Write to CSV
        with open(csv_filepath, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Metric', 'Value'])
            for key, value in results.items():
                writer.writerow([key, value])

        logger.info(f"Simulation results written to: {csv_filepath}")
        return csv_filepath
    def run_simulation(self, total_tasks: int, distribution_type: str = 'default') -> Dict:
        """
        Run the simulation with enhanced logging and timing metrics.

        This method handles the simulation execution, including:
        1. Task distribution via selected algorithm
        2. Real-time processing and visualization
        3. Timing metrics calculation
        4. Resource utilization tracking
        5. Comprehensive result gathering

        Args:
            total_tasks (int): Number of tasks to simulate
            distribution_type (str): The distribution algorithm to use

        Returns:
            Dict: Comprehensive metrics about the simulation run
        """
        # Distribute tasks using the chosen algorithm
        distribution_result = self.distribute_tasks(total_tasks, distribution_type)

        # Check if result contains CSV paths (from hybrid algorithm)
        if isinstance(distribution_result, tuple) and len(distribution_result) == 2:
            tasks, csv_paths = distribution_result
            # Store CSV paths for later analysis
            self.algorithm_tracking_paths = csv_paths
            logger.info(f"Algorithm tracking data saved to: {csv_paths}")
        else:
            tasks = distribution_result
            self.algorithm_tracking_paths = {}

        simulation_start_time = datetime.now().timestamp()

        logger.info(f"Starting simulation with {total_tasks} total tasks")
        logger.info(f"Distributed tasks count: {len(tasks)}")

        # Start with original task distribution logging
        logger.info("\n=== Initial Task Distribution ===")
        for resource in self.resources:
            logger.debug(f"\nProcessing Resource: {resource.type}")
            logger.debug(f"  Current Task: {resource.current_task.type if resource.current_task else 'None'}")
            logger.debug(f"  Queue Length: {len(resource.task_queue)}")
            if resource.task_queue:
                for task in resource.task_queue[:5]:
                    logger.info(f"""
                    Task {task.id}:
                    - Type: {task.type}
                    - CPU Required: {task.total_cpu_required}
                    - Arrival Time: {task.arrival_time}
                    """)

        resource_utilization = {}
        datacenter_utilization_snapshots = []

        # Main simulation loop
        with Live(self.generate_live_feed(), refresh_per_second=1) as live:
            while True:
                self.current_time = datetime.now().timestamp() - simulation_start_time
                all_completed = True

                for resource in self.resources:
                    logger.debug(f"\nProcessing Resource: {resource.type}")
                    logger.debug(f"  Current Task: {resource.current_task.type if resource.current_task else 'None'}")
                    logger.debug(f"  Queue Length: {len(resource.task_queue)}")
                    # Add a debug log right here to check task states
                    for i, task in enumerate(resource.task_queue[:3]):  # Just check first 3 tasks
                        logger.info(f"Queue {i}: Task {task.id} - Status: {task.status}")

                    # Process the current queue for this resource
                    queue_status = resource.process_queue(self.current_time)
                    utilization = resource.calculate_resource_utilization()
                    resource_utilization[resource.type] = utilization

                    # Check completion status
                    if resource.current_task:
                        if (resource.current_task.status != 'COMPLETED' or
                            resource.current_task.completion_percentage < 100):
                            all_completed = False
                    if resource.task_queue:
                        all_completed = False

                # Update datacenter utilization
                datacenter_utilization_snapshots.append({
                    'timestamp': self.current_time,
                    'utilization': self.calculate_datacenter_utilization(
                        simulation_start_time,
                        datetime.now().timestamp() - simulation_start_time
                    )
                })

                # Update live visualization
                live.update(self.generate_live_feed())

                # Check if all tasks are complete
                total_completed = sum(len(r.completed_tasks) for r in self.resources)
                total_failed = sum(len(r.failed_tasks) for r in self.resources) + len(self.metrics['globally_failed_tasks'])

                if all_completed and (total_completed + total_failed >= total_tasks):
                    all_tasks_truly_complete = True

                    # Double-check all tasks are truly complete
                    for resource in self.resources:
                        for task in resource.completed_tasks:
                            if task.completion_percentage < 100 or task.status != 'COMPLETED':
                                all_tasks_truly_complete = False
                                break
                        if not all_tasks_truly_complete:
                            break

                    if all_tasks_truly_complete:
                        break

                time.sleep(0.1)

            # Calculate final metrics
            completed_tasks = []
            for resource in self.resources:
                completed_tasks.extend(resource.completed_tasks)
                logger.info(f"""
                Resource {resource.type} Final Status:
                - Completed Tasks: {len(resource.completed_tasks)}
                - Failed Tasks: {len(resource.failed_tasks)}
                """)

            # Calculate timing metrics with detailed debugging
            total_turnaround_time = 0.0
            total_waiting_time = 0.0
            valid_tasks = 0

            logger.info(f"\nCalculating timing metrics for {len(completed_tasks)} tasks:")

            for task in completed_tasks:
                # Debug log all timing values
                logger.info(f"""
                Task {task.id} Raw Timing Values Debug:
                - Status: {task.status}
                - Arrival Time Present: {task.arrival_time is not None}
                - Arrival Time: {task.arrival_time if task.arrival_time is not None else 'None'}
                - Start Time Present: {task.start_time is not None}
                - Start Time: {task.start_time if task.start_time is not None else 'None'}
                - Completion Time Present: {task.completion_time is not None}
                - Completion Time: {task.completion_time if task.completion_time is not None else 'None'}
                - Actual Exec Time: {task.actual_exec_time}
                """)

                # Check each condition separately for debugging
                has_completion = task.completion_time is not None
                has_arrival = task.arrival_time is not None
                has_exec_time = task.actual_exec_time > 0
                completion_after_arrival = (task.completion_time >= task.arrival_time) if (has_completion and has_arrival) else False

                logger.info(f"""
                Task {task.id} Validation Checks:
                - Has Completion Time: {has_completion}
                - Has Arrival Time: {has_arrival}
                - Has Exec Time > 0: {has_exec_time}
                - Completion After Arrival: {completion_after_arrival}
                """)

                if has_completion and has_arrival and has_exec_time and completion_after_arrival:
                    # Calculate turnaround time
                    task.turnaround_time = self.calculate_turnaround_time(task, simulation_start_time)
                    task.waiting_time = self.calculate_waiting_time(task)

                    # Log task timing details
                    logger.info(f"""
                    Task {task.id} Timing Details:
                    - Arrival Time: {task.arrival_time}
                    - Completion Time: {task.completion_time}
                    - Actual Execution Time: {task.actual_exec_time}
                    - Turnaround Time: {task.turnaround_time}
                    - Waiting Time: {task.waiting_time}
                    """)

                    # Additional validation for calculated times
                    if task.turnaround_time > 0:
                        total_turnaround_time += task.turnaround_time
                        total_waiting_time += task.waiting_time
                        valid_tasks += 1

                        logger.info(f"Task {task.id} ACCEPTED for metrics calculation")
                    else:
                        logger.warning(f"""
                        Task {task.id} has invalid calculated times:
                        - Turnaround Time: {task.turnaround_time:.2f}
                        - Waiting Time: {task.waiting_time:.2f}
                        """)
                else:
                    logger.warning(f"""
                    Task {task.id} failed validation:
                    - Missing completion time: {not has_completion}
                    - Missing arrival time: {not has_arrival}
                    - No execution time: {not has_exec_time}
                    - Completion before arrival: {not completion_after_arrival}
                    """)

            # After processing all tasks, log summary
            logger.info(f"""
            Timing Metrics Summary:
            Total Completed Tasks: {len(completed_tasks)}
            Valid Tasks for Timing: {valid_tasks}
            Total Turnaround Time: {total_turnaround_time:.2f}
            Total Waiting Time: {total_waiting_time:.2f}
            """)

            # Calculate averages
            if valid_tasks > 0:
                avg_turnaround = total_turnaround_time / valid_tasks
                avg_waiting = total_waiting_time / valid_tasks
                logger.info(f"""
                Final Timing Metrics:
                - Valid Tasks: {valid_tasks}
                - Average Turnaround Time: {avg_turnaround:.2f}s
                - Average Waiting Time: {avg_waiting:.2f}s
                """)
            else:
                logger.warning("No valid tasks found for timing calculation!")
                avg_turnaround = 0
                avg_waiting = 0

            # Calculate edge-cloud distribution based on final task assignments
            edge_tasks = 0
            cloud_tasks = 0

            for resource in self.resources:
                if resource.type.startswith("Cloud_"):
                    cloud_tasks += len(resource.completed_tasks) + len(resource.task_queue) + len(resource.failed_tasks)
                else:  # Edge resources (Raspberry Pi, Smartphone)
                    edge_tasks += len(resource.completed_tasks) + len(resource.task_queue) + len(resource.failed_tasks)

            total_assigned = edge_tasks + cloud_tasks

            edge_cloud_distribution = {
                "edge_tasks": edge_tasks,
                "cloud_tasks": cloud_tasks,
                "edge_percentage": (edge_tasks / total_assigned * 100) if total_assigned > 0 else 0,
                "cloud_percentage": (cloud_tasks / total_assigned * 100) if total_assigned > 0 else 0,
                "total_tasks": total_assigned
            }

            logger.info(f"\nEdge-Cloud Distribution for {distribution_type}:")
            logger.info(f"Edge: {edge_tasks} tasks ({edge_cloud_distribution['edge_percentage']:.2f}%)")
            logger.info(f"Cloud: {cloud_tasks} tasks ({edge_cloud_distribution['cloud_percentage']:.2f}%)")

            # Calculate load balancing metrics
            load_balancing_metrics = self.calculate_load_balancing_metrics()

            # Update metrics dictionary
            self.metrics.update({
                'total_tasks': total_tasks,
                'completed_tasks': total_completed,
                'failed_tasks': total_failed,
                'makespan': self.current_time,
                'throughput': total_completed / self.current_time if self.current_time > 0 else 0,
                'load_balancing_metrics': load_balancing_metrics,  # Add load balancing metrics here
                'average_turnaround_time': avg_turnaround,
                'average_waiting_time': avg_waiting,
                'valid_tasks': valid_tasks,
                'edge_cloud_distribution': edge_cloud_distribution  # Add edge-cloud distribution metrics
            })

            # If we have algorithm tracking data, add it to metrics
            if hasattr(self, 'algorithm_tracking_paths') and self.algorithm_tracking_paths:
                self.metrics['algorithm_tracking_paths'] = self.algorithm_tracking_paths

            # Process failures
            detailed_failed_tasks = []
            failed_tasks_by_type = {}
            failed_tasks_by_resource = {}

            for resource in self.resources:
                for task in resource.failed_tasks:
                    failure_info = {
                        'task_id': task.id,
                        'task_type': task.type,
                        'resource_type': resource.type,
                        'reason': getattr(task, 'failure_reason', 'Unknown reason')
                    }
                    detailed_failed_tasks.append(failure_info)
                    failed_tasks_by_type[task.type] = failed_tasks_by_type.get(task.type, 0) + 1
                    failed_tasks_by_resource[resource.type] = failed_tasks_by_resource.get(resource.type, 0) + 1

            # Add globally failed tasks
            for task in self.metrics['globally_failed_tasks']:
                detailed_failed_tasks.append({
                    'task_id': task.id,
                    'task_type': task.type,
                    'resource_type': "None",
                    'reason': getattr(task, 'failure_reason', 'Unknown reason')
                })

            # Update failure metrics
            self.metrics.update({
                'detailed_failed_tasks': detailed_failed_tasks,
                'failed_tasks_by_type': failed_tasks_by_type,
                'failed_tasks_by_resource': failed_tasks_by_resource
            })

            # Store final datacenter utilization
            self.metrics['final_datacenter_utilization'] = datacenter_utilization_snapshots[-1]['utilization'] if datacenter_utilization_snapshots else {}

            # Write results and finish
            results_file = self.write_simulation_results(self.metrics, simulation_start_time, distribution_type)
            logger.info(f"Simulation results saved to: {results_file}")

            # If we have algorithm tracking data, generate the visualizations
            if 'algorithm_tracking_paths' in self.metrics:
                try:
                    visualization_files = self.analyze_algorithm_progress(self.metrics['algorithm_tracking_paths'])
                    logger.info(f"Algorithm progress visualizations saved to: {visualization_files}")
                    self.metrics['algorithm_visualizations'] = visualization_files
                except Exception as e:
                    logger.error(f"Error generating algorithm visualizations: {e}")

            return self.metrics
def create_resources():
    """
    Create resources with 10 Smartphones, 5 Raspberry Pis, and 5 Cloud Hosts
    """
    resources = []

    # Create 10 Smartphone Nodes
    for i in range(1, 11):
        resources.append(
            Resource(
                resource_id=i,
                resource_type=f"Smartphone_{i}",  # Changed from Edge_ to Smartphone_
                cpu_rating=400000,   # 400,000 MI/s
                memory=4,            # 4 GB
                bandwidth=400         # 400 MB/s
            )
        )

    # Create 5 Raspberry Pi Nodes
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+10,  # IDs 11-15
                resource_type=f"Raspberry_{i}",
                cpu_rating=80000,    # 80,000 MI/s
                memory=1,            # 1 GB
                bandwidth=100          # 100 MB/s
            )
        )

    # Create 5 Cloud Hosts
    for i in range(1, 6):
        resources.append(
            Resource(
                resource_id=i+15,  # IDs 16-20
                resource_type=f"Cloud_{i}",
                cpu_rating=1000000,  # 1,000,000 MI/s
                memory=32,           # 32 GB
                bandwidth=1200         # 1200 MB/s
            )
        )

    # Log created resources
    logger.info(f"Created {len(resources)} resources: {[r.type for r in resources]}")

    return resources
def main():

    # Disable all logging at the beginning
    #import logging
    logging.disable(logging.CRITICAL)
    # Create resources
    resources = create_resources()

    # Initialize scheduler
    scheduler = ResourceFocusedScheduler(resources)

    try:
        # Define the total number of tasks - use a very small number for debugging
        total_tasks = 2000  # Just 5 tasks for easier debugging

        # Increase logging detail
        #logger.setLevel(logging.DEBUG)

        logger.info("=== STARTING SIMULATION WITH DETAILED DEBUGGING ===")

        # Run simulation
        metrics = scheduler.run_simulation(total_tasks, distribution_type='hybrid')

        logger.info("=== SIMULATION COMPLETED ===")

        # Print final metrics
        print("\nFinal Simulation Metrics:")
        for key, value in metrics.items():
            if key not in ['resource_utilization', 'datacenter_utilization_snapshots', 'detailed_failed_tasks']:
                print(f"{key}: {value}")

    except Exception as e:
        logger.error(f"Unhandled error in main: {e}")
        import traceback
        traceback.print_exc()
if __name__ == "__main__":
    main()